In [1]:
weights = np.cos(np.deg2rad(lat))
print(weights)

NameError: name 'np' is not defined

In [1]:
#===================================================================================================
# Main calculations - for SpEs etc, and crit freq
#===================================================================================================


#Before running, check:
#    - Set up for correct run name / file locs etc
#    - ds_months & Monthstr are set to the correct months
#    - (lev_sl_idx_min/max are set to chosen heights) N/B. some of the [plotting code](?) is set to 25 levs rather than responding to if the no of sliced levs changes
#    - Set Time parameters 
#            - (search "**Set time parameters**" to find)  e.g. time_ind_2wk_min/max_val, time_ind_2wk_min/max_val_temp, time_it_shape (e.g. 2x 2wk time samples) 
#            - Check times_str_min/max are set to the correct array size for the no of months in dataset

#===================================================================================================

import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
from cftime import datetime 
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
jet = mcm.get_cmap('jet')
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time

import line_profiler
%load_ext line_profiler

%matplotlib inline 

#===================================================================================================
start_time = time.process_time() 
#===================================================================================================

crit_freq_on = 0

#ds_months = ['12','01','02']
#Monthstr = [ 'Dec', 'Jan', 'Feb' ]

ds_months = ['06','07','08']
Monthstr = [ 'Jun', 'Jul', 'Aug' ]

#ds_months = ['07']
#Monthstr = ['Jul']

Monthfolderstr = Monthstr[0] + '-' + Monthstr[-1]
#Monthfolderstr = 'Jun-Aug'

ds_months_shape = len(ds_months)  #3
ds_months_ar = np.arange(0,ds_months_shape)   #[0,1,2]

#===================================================================================================
#**Set time parameters**

#No of time samples to iterate over (e.g. 2x 2 week periods)
time_it_shape = 2   

#--------------------------------------------------------------------------------------------------

time_ind_2wk_min = np.ndarray((time_it_shape), dtype = int)
time_ind_2wk_max = np.ndarray((time_it_shape), dtype = int)
time_ar_2wk = np.arange(0,time_it_shape)


times_str_min = np.array( (time_ar_2wk,time_ar_2wk,time_ar_2wk) , dtype=str  ) 
times_str_max = np.array( (time_ar_2wk,time_ar_2wk,time_ar_2wk) , dtype=str  ) 
#times_str_min = np.array( (time_ar_2wk,time_ar_2wk) , dtype=str  ) 
#times_str_max = np.array( (time_ar_2wk,time_ar_2wk) , dtype=str  ) 
#times_str_min = np.array( (time_ar_2wk) , dtype=str  ) 
#times_str_max = np.array( (time_ar_2wk) , dtype=str  ) 
#--------------------------------------------------------------------------------------------------

lon_shape = 144
lonar = np.arange(0,144)        

lat_shape = 96
latar= np.arange(0,96)

newlat = np.arange(-89.5,90.5,1)
newlat_shape = 180
weights_newlat = np.cos(np.deg2rad(newlat))

intlat = np.arange(-87.5,92.5,5)
intlat_shape = 36

# Slice arrays (lev, altitude) between chosen range
lev_sl_idx_min = 42
lev_sl_idx_max = 66
lev_shape = (lev_sl_idx_max-lev_sl_idx_min)+1
levar = np.arange(0,lev_shape)

#**Set time parameters**
#No of timesteps in one time sample (e.g. two week period)
time_ind_2wk_min_val = 0
time_ind_2wk_max_val = 335
time_shape = (time_ind_2wk_max_val-time_ind_2wk_min_val)+1

#===================================================================================================
#Mptdens_w_sh = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) #(25, 336, 96, 144, 2, 3)
#Mptdens_avv1_b_w = np.empty((lev_shape, newlat_shape, time_it_shape, ds_months_shape), dtype=float) 


Mptdens_sh = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) #(25, 336, 96, 144, 2, 3)
#max_Mptdens_avg2 = np.ndarray((time_it_shape, ds_months_shape), dtype = float)
#argmax_Mptdens_avg2 = np.ndarray((time_it_shape, ds_months_shape), dtype = int)
#Mptdens_std_peak = np.ndarray((lev_shape,time_it_shape, ds_months_shape), dtype = float)
#std_max_Mptdens = np.ndarray((time_it_shape, ds_months_shape), dtype = float)
Mptdens_diff = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 

SpEs_sh_nan = np.empty((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape))
SpEs_sh_nan[:] = np.NaN

Mptdens_avg_temp = np.empty((lev_shape, lat_shape, 3, time_it_shape, ds_months_shape), dtype=float)  #10 min bins (144 lon) -> 30 min bins (48 lon)
Mptdens_avg_b = np.empty((lev_shape, lat_shape, 48, time_it_shape, ds_months_shape), dtype=float)    #10 min bins (144 lon) -> 30 min bins (48 lon)
#-------------------------------------------------
Mptdens_avv1_b = np.empty((lev_shape, newlat_shape, time_it_shape, ds_months_shape), dtype=float) 
Mptdens_avv1_b_5d = np.empty((lev_shape, intlat_shape, time_it_shape, ds_months_shape), dtype=float) 
max_Mptdens_avv1_b_5d = np.empty((intlat_shape, time_it_shape, ds_months_shape), dtype=float) 
max_Mptdens_avv1b5d_l = np.empty((lat_shape, time_it_shape, ds_months_shape), dtype=float)            
Mptdens_avv1b5d_l = np.empty((lev_shape, lat_shape, time_it_shape, ds_months_shape), dtype=float) 

#-------------------------------------------------
Mptdens_avg_bb = np.empty((lev_shape, newlat_shape, 48, time_it_shape, ds_months_shape), dtype=float)  
#Mptdens_avg_bb_w = np.empty((lev_shape, newlat_shape, 48, time_it_shape, ds_months_shape), dtype=float)

Mptdens_avg_bb_5d = np.empty(  (lev_shape,intlat_shape, 48, time_it_shape, ds_months_shape )   , dtype=float )  
Mptdens_avg_bb_5d_avg = np.empty((lev_shape, intlat_shape, 48, ds_months_shape), dtype=float)   
Mptdens_avg_bb_5d_avglev = np.empty((intlat_shape, 48, ds_months_shape), dtype=float)  
Mptdens_avg_bb_5d_dsavg = np.ndarray((lev_shape, intlat_shape, 48), dtype = float)   #Dataset avg at each height/lat/lon
Mptdens_avg_bb_5d_dsavglev = np.ndarray((intlat_shape, 48), dtype = float) #Dataset avg over all levs

if crit_freq_on==1:
    #edens_w_sh = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) #(25, 336, 96, 144, 2, 3)
    
    edens_sh = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) #(25, 336, 96, 144, 2, 3)
    SpEs_e = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
    maxSpEs__e = np.ndarray((time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
    maxSpEs_e = np.ndarray((time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
    foEs__m = np.ndarray((time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
    foEs_m = np.ndarray((time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
    foEs_m_av = np.ndarray((lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
    foEs_m_av_mth = np.ndarray((lat_shape, lon_shape, ds_months_shape), dtype = float)
    foEs_m_av_ds = np.ndarray((lat_shape, lon_shape), dtype = float)

SpEs = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
SpEs_freq_bool = np.empty((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape)) 
SpEs_freq = np.empty((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape))
SpEs_freq_time = np.empty((lev_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape)) 
SpEs_Occ_Freq = np.ndarray((lev_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)   
SpEs_Occ_Freq_temp = np.empty((lev_shape, lat_shape, 3, time_it_shape, ds_months_shape), dtype=float)  #10 min bins (144 lon) -> 30 min bins (48 lon)
#-------------------------------------------------
SpEs_Occ_Fr_b = np.empty((lev_shape, lat_shape, 48, time_it_shape, ds_months_shape), dtype=float)    #10 min bins (144 lon) -> 30 min bins (48 lon)
SpEs_Occ_Fr_b_avg = np.ndarray((lev_shape, lat_shape, 48, ds_months_shape), dtype = float)  #Monthly avg at each height/lat/lon
SpEs_Occ_Fr_b_avglev = np.ndarray((lat_shape, 48, ds_months_shape), dtype = float)  #Monthly avg over all levs
SpEs_Occ_Fr_b_dsavg = np.ndarray((lev_shape, lat_shape, 48), dtype = float)   #Dataset avg at each height/lat/lon
SpEs_Occ_Fr_b_dsavglev = np.ndarray((lat_shape, 48), dtype = float) #Dataset avg over all levs
#-------------------------------------------------
SpEs_Occ_Fr_bb = np.empty((lev_shape, newlat_shape, 48, time_it_shape, ds_months_shape), dtype=float) 
#SpEs_Occ_Fr_bb_w = np.empty((lev_shape, newlat_shape, 48, time_it_shape, ds_months_shape), dtype=float) 

SpEs_Occ_Fr_bb_5d = np.empty(  (lev_shape,intlat_shape, 48, time_it_shape, ds_months_shape )   , dtype=float )  
SpEs_Occ_Fr_bb_5d_avg = np.empty((lev_shape, intlat_shape, 48, ds_months_shape), dtype=float)   
SpEs_Occ_Fr_bb_5d_avglev = np.empty((intlat_shape, 48, ds_months_shape), dtype=float)   
SpEs_Occ_Fr_bb_5d_dsavg = np.ndarray((lev_shape, intlat_shape, 48), dtype = float)   #Dataset avg at each height/lat/lon
SpEs_Occ_Fr_bb_5d_dsavglev = np.ndarray((intlat_shape, 48), dtype = float) #Dataset avg over all levs

#===================================================================================================
#===================================================================================================


for ids in ds_months_ar:
    #_-_-_-_-_-_-_-_-_-_-_-_-JianFei WACCMX Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    run_name = 'Jianfei_run'
    
    file1name ='Nc_Files/Jianfei_WACCMX_files/waccmx_Fe_Fep_' + ds_months[ids] + '.nc'
    #file1name ='Nc_Files/Jianfei_WACCMX_files/waccmx_Na_Nap_' + ds_months[ids] + '.nc'
    M = file1name[-9:]
    Metal=M[0:2]
    ds = xr.open_dataset(file1name)
    
    file2name='Nc_Files/Jianfei_WACCMX_files/waccmx_Z3_T_e_' + ds_months[ids] + '.nc' 
    ds2 = xr.open_dataset(file2name) 
    
    #_-_-_-_-_-_-_-_-_-_-_-Wuhu Ion Transport Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    #run_name = 'Wuhu_IonTr_run'
    ##file1name='Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-'  + ds_months[ids] +  '-*.nc' 
    #run_name = 'Wuhu_IonTr_run_6m'
    #file1name='Nc_Files/CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_Si_Ca_K_iontransport/CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_Si_Ca_K_iontransport.cam.h2.0001-'  + ds_months[ids] +  '-*.nc' 
    ##print(file1name)
    #Metal = 'Si'
    #ds = xr.open_mfdataset(file1name)
    #_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    
    print('Dataset: Month ' + str(ds_months[ids]) )

    timee = ds.variables['time']
    start_cftime_date = '0001-' + str(ds_months[ids]) + '-01'
    times = xr.cftime_range(start=start_cftime_date, periods=672, freq="1H", calendar="noleap")    
    #times2 = ds["time"].dt.strftime("%Y%m%d %H:%M")

    lon = ds.variables['lon']
    lat = ds.variables['lat']
    lev = ds.variables['lev']
    dst = ds.transpose("lev", ...)

    #_-_-_-_-_-_-_-_-_-_-_-JianFei WACCMX Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    ds2t = ds2.transpose("lev", ...)
    temp = ds2t.variables['T']
    geopH = ds2t.variables['Z3'] / 1000 #m-> km
    
    if crit_freq_on==1:
        elect = ds2t.variables['e']
    
    #_-_-_-_-_-_-_-_-_-_-Wuhu Ion Transport Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    #temp = dst.variables['T']
    #elect = dst.variables['e']
    #geopH = dst.variables['Z3'] / 1000 #m-> km
    #_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    
    #Mt = dst.variables[str(Metal)]
    Mpt = dst.variables[str(Metal)+'p']    
    Zavg = geopH.mean(('time','lat', 'lon'))

    #===================================================================================================
    # Slice arrays (lev & alt) between chosen range
    lev_sl = lev[lev_sl_idx_min:lev_sl_idx_max+1]

    lev_sl_ind_min = lev[lev_sl_idx_min] 
    lev_sl_ind_max = lev[lev_sl_idx_max] 
    lev_sl_str = str("%.1e" % lev_sl_ind_max) + 'hPa : ' + str("%.1e" % lev_sl_ind_min) + 'hPa'
    
    Zavg_sl = Zavg[lev_sl_idx_min:lev_sl_idx_max+1]
    Zavg_sl_min = Zavg[lev_sl_idx_min] 
    Zavg_sl_max = Zavg[lev_sl_idx_max]
    Zavg_sl_str = str("%.0f" % Zavg_sl_max) + 'km : ' + str("%.0f" % Zavg_sl_min) + 'km'
    
    print('    Array lev = ' + lev_sl_str + ' (approx ' + Zavg_sl_str + ')'   )

    #geopH_sl = geopH[lev_sl_idx_min:lev_sl_idx_max+1,:,:,:]
   
    #===================================================================================================

    #No of timesteps in one time sample (e.g. two week period)
    time_ind_2wk_min_val_temp = 0
    time_ind_2wk_max_val_temp = 335
    
    #===================================================================================================
    weights = np.cos(np.deg2rad(lat))
    #weights.name = "weights" #doesn't work..
    #print(weights)
    #===================================================================================================

    for it2 in time_ar_2wk:
        nadd = 336
        time_ind_2wk_min[it2] = time_ind_2wk_min_val_temp 
        time_ind_2wk_max[it2] = time_ind_2wk_max_val_temp

        time_ind_2wk_min_val_temp = time_ind_2wk_min_val_temp + nadd
        time_ind_2wk_max_val_temp = time_ind_2wk_max_val_temp + nadd

        #===================================================================================================
        
        # Slice arrays (time) between chosen range & generate strings for figure labelling
        #Time slice 1: 0:335 = 0001-02-01 00:00:00 : 0001-02-14 23:00:00 #336
        #Time slice 2: 336:671 = 0001-02-15 00:00:00 : 0001-02-28 23:00:00 #336

        times_idx_min = time_ind_2wk_min[it2]
        times_idx_max = time_ind_2wk_max[it2]
        times_min = times[times_idx_min]
        times_max = times[times_idx_max]

        if ds_months_shape>1:
            times_str_min[ids][it2] = str(times_min) #~~#
            times_str_max[ids][it2] = str(times_max) #~~#
            print( '      Time slice ' + str(it2+1) + ' = ' +times_str_min[ids][it2] + ' : ' + times_str_max[ids][it2] ) 
        else:
            times_str_min[it2] = str(times_min) #~~#
            times_str_max[it2] = str(times_max) #~~#
            print( '      Time slice ' + str(it2+1) + ' = ' +times_str_min[it2] + ' : ' + times_str_max[it2] ) 

        #===================================================================================================
        
        # Slice temp by chosen alt range and time range 
        tempe = temp[lev_sl_idx_min:lev_sl_idx_max+1,times_idx_min:times_idx_max+1,:,:]
        Mp_t = Mpt[lev_sl_idx_min:lev_sl_idx_max+1,times_idx_min:times_idx_max+1,:,:]
        
        if crit_freq_on==1:
            elec = elect[lev_sl_idx_min:lev_sl_idx_max+1,times_idx_min:times_idx_max+1,:,:]
        
        #===================================================================================================

        # VMR to number density calculation
        Mptdens = ( Mp_t * 1e-6 * 100 * lev_sl ) / (1.380503e-23 * tempe) #Mptdens.shape = (25, 336, 96, 144)
        
        if crit_freq_on==1:
            edens = ( elec * 1e-6 * 100 * lev_sl ) / (1.380503e-23 * tempe)
        
        #Mptdens_w = Mptdens * weights
        #if crit_freq_on==1:
        #    edens_w = edens * weights
        
        #===================================================================================================

        # Create array with time info for each day 
        day = np.arange(0,24)

        # Create an array with offset needed for each UT time step (24h period)
        # Offset by 15 degrees lon, lon axis is in 2.5 degree intervals
        os =  day * 15 / 2.5 
        offset = np.empty(24, dtype=int) #Create offset array so it has same dims as day variable
        for d in day:
            offset[d] =  ( int(os[d]) ) #convert floats to ints to get integer number of indices to offset by

        # Stack multiple offset arrays to give offsets for consecutive days in the 2 week dataset 
        offset2 = np.stack((offset, offset, offset, offset, offset, offset, offset), axis=0)   #1wk
        offset2 = np.reshape(offset2, (24*7))
        offset3 = np.stack((offset2, offset2), axis=0)  #2wks
        offset3 = np.reshape(offset3, (168*2))

        #===================================================================================================
        #loop_start_time = time.process_time()
        
        #Iterate over timear, offsetting e density at each time by 15 degrees lon
        timear = np.arange(0,336) # 2 week iteration period - hourly data
        for it in timear:
            Mptdens_sh[:,it,:,:,it2,ids] = np.roll(Mptdens[:,it,:,:], offset3[it] , axis=2)
            if crit_freq_on==1:
                edens_sh[:,it,:,:,it2,ids] = np.roll(edens[:,it,:,:], offset3[it] , axis=2)
        
        #loop_end_time = time.process_time()
        #loop_time_taken = loop_end_time - loop_start_time
        #loop_time_taken_min = loop_time_taken / 60.
        #print('Loop 2 Time = ' + str(loop_time_taken_min) + ' mins' ) 

        #===================================================================================================
        
        # Calculate average of offset densities along time axis for 2wk sample
        Mptdens_avg = np.mean(Mptdens_sh,  axis=1) #-> (25, 96, 144, 2, 3)

        # Calculate std dev of offset densities along time axis
        Mptdens_std = np.std(Mptdens_sh, axis=1) #-> (25, 96, 144, 2, 3)

        #===================================================================================================
        #Crit_B pre-June 22
        ##Calculate global average of Mp dens as fct of lev, find max over lev dimension (peak of layer)
        ##Calculate standard deviation of this global average layer profile over time dim, and also at max of layer
        #
        #Mptdens_avg0 =  np.mean(Mptdens_sh,  axis=2) #avg over lat dim -> (25, 336, 144, 2, 3)
        #Mptdens_avg1 = np.mean(Mptdens_avg0, axis=2) #avg over lon dim -> (25, 336, 2, 3)
        #Mptdens_avg2 = np.mean(Mptdens_avg1, axis=1) #avg over time dim -> (25, 2, 3) #GLOBAL AVG AT HEIGHT X
        #
        #max_Mptdens_avg2[it2,ids] = np.amax(Mptdens_avg2[:,it2,ids], axis=0) # find max over lev dim  #PEAK OF GLOBAL AVG M LAYER#
        #
        #max_Mptdens_avg2_print = np.round_(max_Mptdens_avg2[it2,ids], decimals=1) 
        #print('          Global avg Metal layer peak = ' + str(max_Mptdens_avg2_print) + ' cm-3' ) #1205.80343702 for it2=0,ds=Feb
        #
        #argmax_Mptdens_avg2[it2,ids] = np.argmax(Mptdens_avg2[:,it2,ids], axis=0) # find index of max over lev dim
        #
        ##print('        Index of global avg Metal layer peak = ' + str(argmax_Mptdens_avg2[it2,ids])) #index 16 for it2=0,ds=Feb
        #lev_sl_print = lev_sl[argmax_Mptdens_avg2[it2,ids]].to_numpy()
        #lev_sl_printt = "{:.2e}".format(lev_sl_print)
        #Zavg_sl_print = Zavg_sl[argmax_Mptdens_avg2[it2,ids]].to_numpy()
        #Zavg_sl_printt = "{:.1f}".format(Zavg_sl_print)
        #print('          Global avg Metal layer peak height = ' + str(lev_sl_printt) + 'hPa (~' + str(Zavg_sl_printt) + 'km)' ) #index 16 for it2=0,ds=Feb
        #
        ##Std dev of global average layer over time dim
        #Mptdens_std_peak[:,it2,ids] = np.std(Mptdens_avg1[:,:,it2,ids], axis=1) #find std dev of Mptdens_avg1 (dims lev, time, it2, ids) over time dim ->(25,2,3)
        #std_max_Mptdens[it2,ids] = Mptdens_std_peak[ argmax_Mptdens_avg2[it2,ids], it2, ids ] #std dev at layer peak
        #
        #std_max_Mptdens_print = np.round_(std_max_Mptdens[it2,ids], decimals=1) 
        #print('          Std Dev of global avg Metal layer peak = ' + str(std_max_Mptdens_print) + ' cm-3' ) #161.189181 for it2=0,ds=Feb
        # 
        #===================================================================================================
        
        #Works out zonal avg M layer in 5' lat slices - for each 2wk time slic 
        #average Mptdens over timestep and lon dims
        Mptdens_avv0 =  np.mean(Mptdens_sh,  axis=1) #avg over time dim -> (25, 96, 144, 2, 3)
        Mptdens_avv1 = np.mean(Mptdens_avv0, axis=2) #avg over lon dim -> (25, 96, 2, 3)
        #- - - - - - - - - - - - - - - - - - - - - - - - - - - 
        # Bin into 1' lat bins instead of 1.89' (180 long) then avg in 5' slices
        intlat_ar = np.arange(0,36)  # array 0->35
        for ilev in levar:
            Mptdens_avv1_b[ilev,:,it2,ids] = np.interp( newlat, lat, Mptdens_avv1[ilev,:,it2,ids] )  #interpolate onto newlat grid 1' spacing
            #Mptdens_avv1_b_w[ilev,:,it2,ids] = Mptdens_avv1_b[ilev,:,it2,ids] * weights_newlat
            
            for iintlat in intlat_ar:
                Mptdens_avv1_b_5d[ilev,iintlat,it2,ids] = np.mean( Mptdens_avv1_b[ilev,(iintlat*5):((iintlat*5)+5),it2,ids] ) #average in 5' lat slices #GLOBAL AVG AT HEIGHT X IN 5' SLICES#
        max_Mptdens_avv1_b_5d[:,it2,ids] = np.amax(Mptdens_avv1_b_5d[:,:,it2,ids], axis=0) # find max over lev dim  #PEAK OF GLOBAL AVG M LAYER IN 5' SLICES#      
        
        #Assigns correct lat slice (dim 26 long) to variable with normal lat axis (96 long) so this can be used in criteria below
        Z = 0
        X = -90
        Y = -85
        for ilat in latar:
            if lat[ilat] > Y :
                X = X + 5
                Y = Y + 5 
                Z = Z + 1
            if (lat[ilat]>=X) & (lat[ilat]<=Y) :
                max_Mptdens_avv1b5d_l[ ilat ,it2,ids] = max_Mptdens_avv1_b_5d[ Z ,it2,ids]
                Mptdens_avv1b5d_l[:, ilat ,it2,ids] = Mptdens_avv1_b_5d[:, Z ,it2,ids]
        
        #===================================================================================================
        
        # PREV CRITERIA TESTED:
            # SpEs_sh_1: (diff > x sigma) 
            # SpEs_sh: (diff > x sigma) & Mpt > glb average at height x
            # SpEs_sh_mx: ** (diff > x sigma) & (Mpt > glb average at height peak of glb avg layer) ** 
            # SpEs_sh_mx1: (Mpt > glb average at height peak of glb avg layer) 
            # SpEs_sh_mx2: (diff > 0.5 sigma) & (Mpt > glb average at height peak of glb avg layer)

        #(A) ( Diff > x sigma ) & ( Mpt > 3x glb average at height x ) & ( Mpt > 3x std dev at glb avg layer peak)  
        #(B) ( Diff > x sigma ) & ( Mpt > 3x glb average at height x ) & ( Mpt > 1x glb avg layer peak)   
        #(C) ( Diff > x sigma ) & ( Mpt > 3* grid box avg over time period ) & ( Mpt > 1x glb avg layer peak)
        #(D) ( Diff > x sigma ) & ( Mpt > 3* grid box avg over time period ) & ( Mpt > 3x std dev at glb avg layer peak)
        #(E) ( Diff > x sigma ) & ( Mpt > 3x glb average at height x )  & ( Mpt > 1* grid box avg over time period ) & ( Mpt > 1x glb avg layer peak) 
        #(F) ( Mpt > 3x glb average at height x ) & ( Mpt > 3x std dev at glb avg layer peak) 

        #===================================================================================================

        # Calculate difference between e density from model output and the average e density
        for it in timear:
            Mptdens_diff[:,it,:,:,it2,ids] = Mptdens_sh[:,it,:,:,it2,ids] - Mptdens_avg[:,:,:,it2,ids]

        #-----------------------------------------------------------------------
        sigma_val = 1   #1sigma~68%   1.5sigma~87%    2sigma~95%
        sigma_val_str = str(sigma_val)
        #-----------------------------------------------------------------------

        # Current criteria:
        #
        Crit_name = '(B)'

        # Where criteria set are met, set SpEs to Mptdens_sh, otherwise set to NaN
        

        
        #Crit_B pre-June 22
        #for it in timear:
        #    for ilev in levar:
        #        SpEs[ilev,it,:,:,it2,ids] = np.where(  ( Mptdens_diff[ilev,it,:,:,it2,ids]>( sigma_val*Mptdens_std[ilev,:,:,it2,ids]) ) &     #( Diff > x sigma )
        #                                         ( Mptdens_sh[ilev,it,:,:,it2,ids] > ( 3. * Mptdens_avg2[ilev,it2,ids] ) ) &          #( Mpt > 3x glb average at height x ) 
        #                                         #( Mptdens_sh[ilev,it,:,:,it2,ids] > ( 3. * Mptdens_avg[ilev,:,:,it2,ids]) ) &         #( Mpt > 3* grid box avg over time period )
        #                                         #( Mptdens_sh[ilev,it,:,:,it2,ids] > ( 1. * Mptdens_avg[ilev,:,:,it2,ids]) ) &         #( Mpt > 1* grid box avg over time period )
        #                                         ( Mptdens_sh[ilev,it,:,:,it2,ids] > ( max_Mptdens_avg2[it2,ids]) )                    #( Mpt > 1x glb avg layer peak)                       
        #                                         #( Mptdens_sh[ilev,it,:,:,it2,ids] > ( 3. * std_max_Mptdens[it2,ids] ) )               #( Mpt > 3x std dev at glb avg layer peak)
        #                                                      , Mptdens_sh[ilev,it,:,:,it2,ids] , SpEs_sh_nan[ilev,it,:,:,it2,ids] )

        #------------------------Crit_B* Using 5' lat slices for global avg--------------------------------       
        #using zonal avg / 5' lat slice for global avg
        for it in timear:
            for ilev in levar:
                for ilat in latar:
                    SpEs[ilev,it,ilat,:,it2,ids] = np.where(  ( Mptdens_diff[ilev,it,ilat,:,it2,ids]>( sigma_val*Mptdens_std[ilev,ilat,:,it2,ids]) ) &     #( Diff > x sigma )   
                                                 ( Mptdens_sh[ilev,it,ilat,:,it2,ids] > ( 3. * Mptdens_avv1b5d_l[ilev,ilat,it2,ids] ) ) &        #( Mpt > 3x glb average at height x in 5' slice) 
                                                 ( Mptdens_sh[ilev,it,ilat,:,it2,ids] > ( max_Mptdens_avv1b5d_l[ilat,it2,ids] ) )         #( Mpt > 1x glb avg layer peak in 5' slice)                       
                                                              , Mptdens_sh[ilev,it,ilat,:,it2,ids] , SpEs_sh_nan[ilev,it,ilat,:,it2,ids] )
                #--------------------------------------------------------
                if crit_freq_on==1:
                    SpEs_e[ilev,it,:,:,it2,ids] = np.where( SpEs[ilev,it,:,:,it2,ids]==Mptdens_sh[ilev,it,:,:,it2,ids], edens_sh[ilev,it,:,:,it2,ids] , SpEs_sh_nan[ilev,it,:,:,it2,ids] ) 
                    
                    
        #===================================================================================================
        if crit_freq_on==1:    
            #Calculate critical ionosonde frequency    
            #avgSpEs__e[:,:,:,it2,ids] = np.nanmean(SpEs_e[:,:,:,:,it2,ids], axis=0)  
            #avgSpEs_e[:,:,:,it2,ids] =avgSpEs__e[:,:,:,it2,ids] * 1e6   #cm-3 -> m-3
            maxSpEs__e[:,:,:,it2,ids] = np.nanmax(SpEs_e[:,:,:,:,it2,ids], axis=0)  
            maxSpEs_e[:,:,:,it2,ids] =maxSpEs__e[:,:,:,it2,ids] * 1e6   #cm-3 -> m-3
            for ilat in latar:
                for ilon in lonar:
                    for it in timear:
                        #foEs__a[it,ilat,ilon,it2,ids] = 8.98 * np.sqrt(avgSpEs_e[it,ilat,ilon,it2,ids])
                        #foEs_a[it,ilat,ilon,it2,ids] = foEs__a[it,ilat,ilon,it2,ids] / 1e6    #Hz -> MHz
                        foEs__m[it,ilat,ilon,it2,ids] = 8.98 * np.sqrt(maxSpEs_e[it,ilat,ilon,it2,ids])
                        foEs_m[it,ilat,ilon,it2,ids] = foEs__m[it,ilat,ilon,it2,ids] / 1e6    #Hz -> MHz

            #foEs_a_av[:,:,it2,ids] = np.nanmean( foEs_a[:,:,:,it2,ids], axis=0 )
            foEs_m_av[:,:,it2,ids] = np.nanmean( foEs_m[:,:,:,it2,ids], axis=0 ) #avg over lev dim
        
        #===================================================================================================
        #Calculate Occurence freq

        SpEs_freq_bool[:,:,:,:,it2,ids] = np.isfinite(SpEs[:,:,:,:,it2,ids]) #Where SpEs is a number, set to True, otherwise set to False
                          
        SpEs_freq[:,:,:,:,it2,ids] = SpEs_freq_bool[:,:,:,:,it2,ids] *1. #Convert True/False to 1/0s -> (25,336,96,144,2,3)
        
        SpEs_freq_time[:,:,:,it2,ids] = np.sum(SpEs_freq[:,:,:,:,it2,ids], axis=1) #Sum over Time dim to give SpEs_sh_freq_time (total occurences in 2 week time period) -> (25,96,144,2,3)
        SpEs_Occ_Freq[:,:,:,it2,ids] = ( SpEs_freq_time[:,:,:,it2,ids] / time_shape ) *100.    #Divide by number of timesteps (336) to give occurence freq (%)  # (25,96,144,2)
          
                 
        #===================================================================================================
        ###_SpEs_Occ_Freq_###
        ###_Mptdens_avg_##    # =Mptdens avged over time dim for 2wk sample
        
        # Bin into 0.5h LT bins instead of 10 mins 

        it_ar = np.arange(1,48) # array 1->47
        #latar= np.arange(0,96)
        for ilev in levar:
            for ilat in latar:
                SpEs_Occ_Freq_temp[ilev,ilat,0,it2,ids] = SpEs_Occ_Freq[ilev,ilat,-1,it2,ids]    # (25,96,144,2,3)
                SpEs_Occ_Freq_temp[ilev,ilat,1,it2,ids] = SpEs_Occ_Freq[ilev,ilat,0,it2,ids]
                SpEs_Occ_Freq_temp[ilev,ilat,2,it2,ids] = SpEs_Occ_Freq[ilev,ilat,1,it2,ids]
                SpEs_Occ_Fr_b[ilev,ilat,0,it2,ids] = np.mean(SpEs_Occ_Freq_temp[ilev,ilat,:,it2,ids])
                
                Mptdens_avg_temp[ilev,ilat,0,it2,ids] = Mptdens_avg[ilev,ilat,-1,it2,ids]    # (25,96,144,2,3)
                Mptdens_avg_temp[ilev,ilat,1,it2,ids] = Mptdens_avg[ilev,ilat,0,it2,ids]
                Mptdens_avg_temp[ilev,ilat,2,it2,ids] = Mptdens_avg[ilev,ilat,1,it2,ids]
                Mptdens_avg_b[ilev,ilat,0,it2,ids] = np.mean(Mptdens_avg_temp[ilev,ilat,:,it2,ids])
                
                for iit_ar in it_ar:
                    SpEs_Occ_Freq_temp[ilev,ilat,:,it2,ids] = SpEs_Occ_Freq[ilev,ilat,((iit_ar*3)-1):((iit_ar*3)+2),it2,ids]
                    SpEs_Occ_Fr_b[ilev,ilat,iit_ar,it2,ids] = np.mean(SpEs_Occ_Freq_temp[ilev,ilat,:,it2,ids])
                    
                    Mptdens_avg_temp[ilev,ilat,:,it2,ids] = Mptdens_avg[ilev,ilat,((iit_ar*3)-1):((iit_ar*3)+2),it2,ids]
                    Mptdens_avg_b[ilev,ilat,iit_ar,it2,ids] = np.mean(Mptdens_avg_temp[ilev,ilat,:,it2,ids])
        
        #- - - - - - - - - - - - - - - - - - - - - - - - - - - 
       
        # Bin into 1' lat bins instead of 1.89' (180 long) then avg in 5' slices
        
        it_arr = np.arange(0,48) # array 0->47
        intlat_ar = np.arange(0,36)  # array 0->35
        
        for ilev in levar:
            for iit_arr in it_arr:
                SpEs_Occ_Fr_bb[ilev,:,iit_arr,it2,ids] = np.interp( newlat, lat, SpEs_Occ_Fr_b[ilev,:,iit_arr,it2,ids] )  #interpolate onto newlar grid 1' spacing
                Mptdens_avg_bb[ilev,:,iit_arr,it2,ids] = np.interp( newlat, lat, Mptdens_avg_b[ilev,:,iit_arr,it2,ids] )  #interpolate onto newlar grid 1' spacing
                
                #SpEs_Occ_Fr_bb_w[ilev,:,iit_arr,it2,ids] = SpEs_Occ_Fr_bb[ilev,:,iit_arr,it2,ids] * weights_newlat
                #Mptdens_avg_bb_w[ilev,:,iit_arr,it2,ids] = Mptdens_avg_bb[ilev,:,iit_arr,it2,ids] * weights_newlat                
                for iintlat in intlat_ar:
                    SpEs_Occ_Fr_bb_5d[ilev,iintlat,iit_arr,it2,ids] = np.mean(   SpEs_Occ_Fr_bb[ilev,(iintlat*5):((iintlat*5)+5),iit_arr,it2,ids] )   #average in 5' slices
                    Mptdens_avg_bb_5d[ilev,iintlat,iit_arr,it2,ids] = np.mean(   Mptdens_avg_bb[ilev,(iintlat*5):((iintlat*5)+5),iit_arr,it2,ids] )   #average in 5' slices
           
        #===================================================================================================
        
    
    #--- Out of it2 loop (2wk time periods), still in ds loop (month) ---

    ###__Mptdens_avg__###
    #5' lat bins
    Mptdens_avg_bb_5d_avg[:,:,:,ids] = np.mean(Mptdens_avg_bb_5d[:,:,:,:,ids],  axis=3) #avg over it2 dim (avg both 2wk periods)   =Monthly avg  
    Mptdens_avg_bb_5d_avglev[:,:,ids] = np.mean(Mptdens_avg_bb_5d_avg[:,:,:,ids],  axis=0) #avg over lev dim   =Monthly avg over all heights  
        
    ###__SpEs_Occ_Fr__###
    #Specific lats
    SpEs_Occ_Fr_b_avg[:,:,:,ids] = np.mean(SpEs_Occ_Fr_b[:,:,:,:,ids],  axis=3) #avg over it2 dim (avg both 2wk periods)   =Monthly avg   ->(25, 96, 48, 3)
    SpEs_Occ_Fr_b_avglev[:,:,ids] = np.mean(SpEs_Occ_Fr_b_avg[:,:,:,ids],  axis=0) #avg over lev dim   =Monthly avg over all heights   ->(96, 48, 3)
    #5' lat bins
    SpEs_Occ_Fr_bb_5d_avg[:,:,:,ids] = np.mean(SpEs_Occ_Fr_bb_5d[:,:,:,:,ids],  axis=3) #avg over it2 dim (avg both 2wk periods)   =Monthly avg   ->(25, 36, 48, 3)
    SpEs_Occ_Fr_bb_5d_avglev[:,:,ids] = np.mean(SpEs_Occ_Fr_bb_5d_avg[:,:,:,ids],  axis=0) #avg over lev dim   =Monthly avg over all heights   ->(36, 48, 3)
    
    
    ###__FoEs_Monthly_avg___###
    if crit_freq_on==1:
        #foEs_a_av_mth[:,:,ids] = np.nanmean( foEs_a_av[:,:,:,ids], axis=2 )
        foEs_m_av_mth[:,:,ids] = np.nanmean( foEs_m_av[:,:,:,ids], axis=2 )
    
    
#--- Out of it2 loop (2wk time periods) AND ds loop (month) ---

###__Mptdens_avg__###
#5' lat bins
Mptdens_avg_bb_5d_dsavg[:,:,:] = np.mean(Mptdens_avg_bb_5d_avg[:,:,:,:],  axis=3) #avg over ids dim (avg all months)   =Whole Dataset avg   ->(25, 36, 48)
Mptdens_avg_bb_5d_dsavglev[:,:] = np.mean(Mptdens_avg_bb_5d_dsavg[:,:,:],  axis=0) #avg over lev dim ->(36, 48)


###__SpEs_Occ_Fr__###
#specific lats
SpEs_Occ_Fr_b_dsavg[:,:,:] = np.mean(SpEs_Occ_Fr_b_avg[:,:,:,:],  axis=3) #avg over ids dim (avg all months)   =Whole Dataset avg   ->(25, 96, 48)
SpEs_Occ_Fr_b_dsavglev[:,:] = np.mean(SpEs_Occ_Fr_b_dsavg[:,:,:],  axis=0) #avg over lev dim ->(96, 48)
#5' lat bins
SpEs_Occ_Fr_bb_5d_dsavg[:,:,:] = np.mean(SpEs_Occ_Fr_bb_5d_avg[:,:,:,:],  axis=3) #avg over ids dim (avg all months)   =Whole Dataset avg   ->(25, 36, 48)
SpEs_Occ_Fr_bb_5d_dsavglev[:,:] = np.mean(SpEs_Occ_Fr_bb_5d_dsavg[:,:,:],  axis=0) #avg over lev dim ->(36, 48)


###__FoEs_ds_avg___###
if crit_freq_on==1:
    #foEs_a_av_ds[:,:] = np.nanmean( foEs_a_av_mth[:,:,:], axis=2 )
    foEs_m_av_ds[:,:] = np.nanmean( foEs_m_av_mth[:,:,:], axis=2 )


#===================================================================================================
end_time = time.process_time()
time_taken = end_time - start_time
time_taken_min = time_taken / 60.
print('========================================')
print('Calculation Time = ' + str(time_taken_min) + ' mins' )  #1 month ~3.3mins #~13 mins 3mths, ~23mins with crit_freq_on 


FileNotFoundError: [Errno 2] No such file or directory: b'/localhome/sestay/jupyter/Old_Notebooks/Nc_Files/Jianfei_WACCMX_files/waccmx_Fe_Fep_06.nc'

In [ ]:
#===================================================================================================
# Main calculations - for Winds
#===================================================================================================

#Before running, check:
#    - Set up for correct run name / file locs etc
#    - ds_months & Monthstr are set to the correct months
#    - (lev_sl_idx_min/max are set to chosen heights) N/B. some of the [plotting code](?) is set to 25 levs rather than responding to if the no of sliced levs changes
#    - Set Time parameters 
#            - (search "**Set time parameters**" to find)  e.g. time_ind_2wk_min/max_val, time_ind_2wk_min/max_val_temp, time_it_shape (e.g. 2x 2wk time samples) 
#            - Check times_str_min/max are set to the correct array size for the no of months in dataset

#===================================================================================================


#===================================================================================================
start_time = time.process_time() 
#===================================================================================================

winds_on = 1

#ds_months = ['12','01','02']
#Monthstr = [ 'Dec', 'Jan', 'Feb' ]

#ds_months = ['06','07','08']
#Monthstr = [ 'Jun', 'Jul', 'Aug' ]

ds_months = ['07']
Monthstr = [ 'Jul']

Monthfolderstr = Monthstr[0] + '-' + Monthstr[-1]
#Monthfolderstr = 'Jun-Aug'

ds_months_shape = len(ds_months)  #3
ds_months_ar = np.arange(0,ds_months_shape)   #[0,1,2]

#===================================================================================================
#**Set time parameters**

#No of time samples to iterate over (e.g. 2x 2 week periods)
time_it_shape = 2   

#--------------------------------------------------------------------------------------------------

time_ind_2wk_min = np.ndarray((time_it_shape), dtype = int)
time_ind_2wk_max = np.ndarray((time_it_shape), dtype = int)
time_ar_2wk = np.arange(0,time_it_shape)


#times_str_min = np.array( (time_ar_2wk,time_ar_2wk,time_ar_2wk) , dtype=str  ) 
#times_str_max = np.array( (time_ar_2wk,time_ar_2wk,time_ar_2wk) , dtype=str  ) 
#times_str_min = np.array( (time_ar_2wk,time_ar_2wk) , dtype=str  ) 
#times_str_max = np.array( (time_ar_2wk,time_ar_2wk) , dtype=str  ) 
times_str_min = np.array( (time_ar_2wk) , dtype=str  ) 
times_str_max = np.array( (time_ar_2wk) , dtype=str  ) 
#--------------------------------------------------------------------------------------------------

lon_shape = 144
lonar = np.arange(0,144)        

lat_shape = 96
latar= np.arange(0,96)

newlat = np.arange(-89.5,90.5,1)
newlat_shape = 180
intlat = np.arange(-87.5,92.5,5)
intlat_shape = 36

# Slice arrays (lev, altitude) between chosen range
lev_sl_idx_min = 42
lev_sl_idx_max = 66
lev_shape = (lev_sl_idx_max-lev_sl_idx_min)+1
levar = np.arange(0,lev_shape)
ilevar = np.arange(0,lev_shape+1)
levarsl = np.arange(0,lev_shape-1)
levarr = np.arange(1,lev_shape+1) #1-->25

#**Set time parameters**
#No of timesteps in one time sample (e.g. two week period)
time_ind_2wk_min_val = 0
time_ind_2wk_max_val = 335
time_shape = (time_ind_2wk_max_val-time_ind_2wk_min_val)+1
timear = np.arange(0,time_shape)
#===================================================================================================


#===================================================================================================
#===================================================================================================

U_wind_sh = np.ndarray( (lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 

U_wind_sl2_sh = np.ndarray((lev_shape+2,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 

dz = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dz_sh = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 

alt_up = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 
alt_low = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 

U_up = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
U_low = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dU = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)  
dUdz = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)

U_wind_sh_2wk = np.ndarray((lev_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
U_wind_sh_mth = np.ndarray((lev_shape, lat_shape, lon_shape, ds_months_shape), dtype = float)
U_wind_sh_ds = np.ndarray((lev_shape, lat_shape, lon_shape), dtype = float)

dUdz_2wk = np.ndarray((lev_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dUdz_mth = np.ndarray((lev_shape, lat_shape, lon_shape, ds_months_shape), dtype = float)
dUdz_ds = np.ndarray((lev_shape, lat_shape, lon_shape), dtype = float)

#V_wind_sh = np.ndarray( (lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
#V_wind_sh_2wk = np.ndarray((lev_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
#V_wind_sh_mth = np.ndarray((lev_shape, lat_shape, lon_shape, ds_months_shape), dtype = float)
#V_wind_sh_ds = np.ndarray((lev_shape, lat_shape, lon_shape), dtype = float)

##V_up = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
##V_low = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
##dV = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)  
##dVdz = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
##V_wind_sl2_sh = np.ndarray((lev_shape+2,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float) 

#dU2V2 = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
##dU2V2dz = np.ndarray((lev_shape,time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)

dUdz_2 = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dUdz_2_NaN = np.empty((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dUdz_2_NaN[:] = np.NaN
#===================================================================================================


for ids in ds_months_ar:
    #_-_-_-_-_-_-_-_-_-_-_-_-JianFei WACCMX Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    run_name = 'Jianfei_run'
    
    file1name ='Nc_Files/Jianfei_WACCMX_files/waccmx_Fe_Fep_' + ds_months[ids] + '.nc'
    #file1name ='Nc_Files/Jianfei_WACCMX_files/waccmx_Na_Nap_' + ds_months[ids] + '.nc'
    #M = file1name[-9:]
    #Metal=M[0:2]
    ds = xr.open_dataset(file1name)
    
    file2name='Nc_Files/Jianfei_WACCMX_files/waccmx_Z3_T_e_' + ds_months[ids] + '.nc' 
    ds2 = xr.open_dataset(file2name) 
    
    #if winds_on == 1:
    file3name ='Nc_Files/Jianfei_WACCMX_files/waccmx_winds_' + ds_months[ids] + '.nc'
    ds3 = xr.open_dataset(file3name)
    
#~    #_-_-_-_-_-_-_-_-_-_-_-Wuhu Ion Transport Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
#~    #run_name = 'Wuhu_IonTr_run'
#~    #file1name='Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-'  + ds_months[ids] +  '-*.nc' 
#~    #Metal = 'Fe'
#~    #ds = xr.open_mfdataset(file1name)
#~    #_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    
    print('Dataset: Month ' + str(ds_months[ids]) )

    timee = ds.variables['time']
    start_cftime_date = '0001-' + str(ds_months[ids]) + '-01'
    times = xr.cftime_range(start=start_cftime_date, periods=672, freq="1H", calendar="noleap")    
    #times2 = ds["time"].dt.strftime("%Y%m%d %H:%M")

    lon = ds.variables['lon']
    lat = ds.variables['lat']
    lev = ds.variables['lev']
    dst = ds.transpose("lev", ...)

    #_-_-_-_-_-_-_-_-_-_-_-JianFei WACCMX Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
    ds2t = ds2.transpose("lev", ...)
    temp = ds2t.variables['T']
    geopH = ds2t.variables['Z3'] / 1000 #m-> km
    
    #if winds_on == 1:
    ds3t = ds3.transpose("lev", ...)
    ilev = ds3t.variables['ilev'] #(127)
    U = ds3t.variables['U'] #(126, 672, 96, 144) #U component is positive for a west to east flow (eastward wind) 
    #V = ds3t.variables['V'] #(126, 672, 96, 144) #V component is positive for south to north flow (northward wind)
    
    
#~    #_-_-_-_-_-_-_-_-_-_-Wuhu Ion Transport Run_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
#~    #temp = dst.variables['T']
#~    #elect = dst.variables['e']
#~    #geopH = dst.variables['Z3'] / 1000 #m-> km
#~    #_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
       
    Zavg = geopH.mean(('time','lat', 'lon'))

    #===================================================================================================
    # Slice arrays (lev & alt) between chosen range (~80-130 km)
    lev_sl = lev[lev_sl_idx_min:lev_sl_idx_max+1]

    lev_sl_ind_min = lev[lev_sl_idx_min] 
    lev_sl_ind_max = lev[lev_sl_idx_max] 
    lev_sl_str = str("%.1e" % lev_sl_ind_max) + 'hPa : ' + str("%.1e" % lev_sl_ind_min) + 'hPa'
    
    Zavg_sl = Zavg[lev_sl_idx_min:lev_sl_idx_max+1]
    Zavg_sl_min = Zavg[lev_sl_idx_min] 
    Zavg_sl_max = Zavg[lev_sl_idx_max]
    Zavg_sl_str = str("%.0f" % Zavg_sl_max) + 'km : ' + str("%.0f" % Zavg_sl_min) + 'km'
    
    print('    Array lev = ' + lev_sl_str + ' (approx ' + Zavg_sl_str + ')'   )

    geopH_sl = geopH[lev_sl_idx_min:lev_sl_idx_max+1,:,:,:]
   
    #===================================================================================================

    #No of timesteps in one time sample (e.g. two week period)
    time_ind_2wk_min_val_temp = 0
    time_ind_2wk_max_val_temp = 335

    for it2 in time_ar_2wk:
        #Create arrays with correct indices for 2x time slices
        nadd = 336
        time_ind_2wk_min[it2] = time_ind_2wk_min_val_temp 
        time_ind_2wk_max[it2] = time_ind_2wk_max_val_temp

        time_ind_2wk_min_val_temp = time_ind_2wk_min_val_temp + nadd
        time_ind_2wk_max_val_temp = time_ind_2wk_max_val_temp + nadd

        #===================================================================================================
        
        # Slice arrays (time) between chosen range
        #Time slice 1: 0:335 = 0001-02-01 00:00:00 : 0001-02-14 23:00:00 #336
        #Time slice 2: 336:671 = 0001-02-15 00:00:00 : 0001-02-28 23:00:00 #336

        times_idx_min = time_ind_2wk_min[it2]
        times_idx_max = time_ind_2wk_max[it2]
        times_min = times[times_idx_min]
        times_max = times[times_idx_max]

        if ds_months_shape>1:
            times_str_min[ids][it2] = str(times_min) #~~#
            times_str_max[ids][it2] = str(times_max) #~~#
            print( '      Time slice ' + str(it2+1) + ' = ' +times_str_min[ids][it2] + ' : ' + times_str_max[ids][it2] ) 
        else:
            times_str_min[it2] = str(times_min) #~~#
            times_str_max[it2] = str(times_max) #~~#
            print( '      Time slice ' + str(it2+1) + ' = ' +times_str_min[it2] + ' : ' + times_str_max[it2] ) 

        #===================================================================================================
        loop_start_time = time.process_time()
        
        
        #print( '1' )
        geopH_sl_time = geopH[:,times_idx_min:times_idx_max+1,:,:] #Just slice by time dim and leave full alt dim
        #print( '2' )
        #slice by one extra either side in lev dim so can calculate dU at array edges
        U_wind_sl2 = U[lev_sl_idx_min-1:lev_sl_idx_max+2,times_idx_min:times_idx_max+1,:,:] #(126, 336, 96, 144)
        #V_wind_sl2 = V[lev_sl_idx_min-1:lev_sl_idx_max+2,times_idx_min:times_idx_max+1,:,:] #(126, 336, 96, 144) 
        #print( '3' )
        geopH_sl = geopH[lev_sl_idx_min:lev_sl_idx_max+1,times_idx_min:times_idx_max+1,:,:]

        # Slice temp by chosen alt range and time range (2 weeks of data)    
        #print( '4' )
        U_wind = U[lev_sl_idx_min:lev_sl_idx_max+1,times_idx_min:times_idx_max+1,:,:] #(25, 336, 96, 144)
        #V_wind = V[lev_sl_idx_min:lev_sl_idx_max+1,times_idx_min:times_idx_max+1,:,:] #(25, 336, 96, 144)

        loop_end_time = time.process_time()
        loop_time_taken = loop_end_time - loop_start_time
        loop_time_taken_min = loop_time_taken / 60.
        print('Loop 1 Time = ' + str(loop_time_taken_min) + ' mins' ) 

        #===================================================================================================

        # Create array with time info for each day 
        day = np.arange(0,24)

        # Create an array with offset needed for each UT time step (24h period)
        # Offset by 15 degrees lon, lon axis is in 2.5 degree intervals
        os =  day * 15 / 2.5 
        offset = np.empty(24, dtype=int) #Create offset array so it has same dims as day variable
        for d in day:
            offset[d] =  ( int(os[d]) ) #convert floats to ints to get integer number of indices to offset by

        # Stack multiple offset arrays to give offsets for consecutive days in the 2 week dataset 
        offset2 = np.stack((offset, offset, offset, offset, offset, offset, offset), axis=0)
        offset2 = np.reshape(offset2, (24*7))
        offset3 = np.stack((offset2, offset2), axis=0)
        offset3 = np.reshape(offset3, (168*2))

        #===================================================================================================
        loop_start_time = time.process_time()
        
        #Iterate over timear, offsetting e density at each time by 15 degrees lon
        timear = np.arange(0,336) # 2 week iteration period - hourly data
        for it in timear:
            U_wind_sh[:,it,:,:,it2,ids] = np.roll(U_wind[:,it,:,:], offset3[it] , axis=2)
            U_wind_sl2_sh[:,it,:,:,it2,ids] = np.roll(U_wind_sl2[:,it,:,:], offset3[it] , axis=2)
            #V_wind_sh[:,it,:,:,it2,ids] = np.roll(V_wind[:,it,:,:], offset3[it] , axis=2)
            #V_wind_sl2_sh[:,it,:,:,it2,ids] = np.roll(V_wind_sl2[:,it,:,:], offset3[it] , axis=2)

        loop_end_time = time.process_time()
        loop_time_taken = loop_end_time - loop_start_time
        loop_time_taken_min = loop_time_taken / 60.
        print('Loop 2 Time = ' + str(loop_time_taken_min) + ' mins' ) 
        
        #===================================================================================================
        loop_start_time = time.process_time()
        
        #~~~~WINDS~~~~
        #if winds_on == 1:
        alt_up[0,:,:,:,it2,ids] = ( geopH_sl_time[lev_sl_idx_min-1,:,:,:] + geopH_sl[0,:,:,:] ) / 2.  #Set alti for first lev index 
        alt_low[0,:,:,:,it2,ids] = ( geopH_sl[0,:,:,:] + geopH_sl[1,:,:,:] ) / 2.
        dz[0,:,:,:,it2,ids] = alt_up[0,:,:,:,it2,ids] - alt_low[0,:,:,:,it2,ids]

        #Set alti for last lev index
        alt_up[24,:,:,:,it2,ids] = ( geopH_sl[23,:,:,:] + geopH_sl[24,:,:,:] ) / 2.
        alt_low[24,:,:,:,it2,ids] = ( geopH_sl[24,:,:,:] + geopH_sl_time[lev_sl_idx_max+1,:,:,:] ) / 2.
        dz[24,:,:,:,it2,ids] = alt_up[24,:,:,:,it2,ids] - alt_low[24,:,:,:,it2,ids]

        levarch = levar[1:-1]
        for ilevch in levarch:
            alt_up[ilevch,:,:,:,it2,ids] = ( geopH_sl[ilevch-1,:,:,:] + geopH_sl[ilevch,:,:,:] ) / 2.
            alt_low[ilevch,:,:,:,it2,ids] = ( geopH_sl[ilevch,:,:,:] + geopH_sl[ilevch+1,:,:,:] ) / 2.
            dz[ilevch,:,:,:,it2,ids] = alt_up[ilevch,:,:,:,it2,ids] - alt_low[ilevch,:,:,:,it2,ids] 
        dz = dz * 1000. #km --> m
        #~~~~~~~~~~~~
        #Convert dz into local time grid--------------------------------------------
        #Iterate over timear, offsetting at each time by 15 degrees lon
        for it in timear:
            dz_sh[:,it,:,:,it2,ids] = np.roll(dz[:,it,:,:,it2,ids], offset3[it] , axis=2)
        #---------------------------------------------------------------------------
        levarr = np.arange(1,lev_shape+1) #1-->25
        for ilev in levarr:
            U_up[ilev-1,:,:,:,it2,ids] = U_wind_sl2_sh[ilev-1,:,:,:,it2,ids] - U_wind_sl2_sh[ilev,:,:,:,it2,ids] #ms-1
            U_low[ilev-1,:,:,:,it2,ids] = U_wind_sl2_sh[ilev,:,:,:,it2,ids] - U_wind_sl2_sh[ilev+1,:,:,:,it2,ids] #ms-1
            dU[ilev-1,:,:,:,it2,ids] = U_up[ilev-1,:,:,:,it2,ids] - U_low[ilev-1,:,:,:,it2,ids] #ms-1
            #V_up[ilev-1,:,:,:,it2,ids] = V_wind_sl2_sh[ilev-1,:,:,:,it2,ids] - V_wind_sl2_sh[ilev,:,:,:,it2,ids] #ms-1
            #V_low[ilev-1,:,:,:,it2,ids] = V_wind_sl2_sh[ilev,:,:,:,it2,ids] - V_wind_sl2_sh[ilev+1,:,:,:,it2,ids] #ms-1
            #dV[ilev-1,:,:,:,it2,ids] = V_up[ilev-1,:,:,:,it2,ids] - V_low[ilev-1,:,:,:,it2,ids] #ms-1

        dUdz[:,:,:,:,it2,ids] = dU[:,:,:,:,it2,ids] / dz_sh[:,:,:,:,it2,ids] #s-1
        #dVdz[:,:,:,:,it2,ids] = dV[:,:,:,:,it2,ids] / dz_sh[:,:,:,:,it2,ids] #s-1

        dUdz_2[:,:,:,:,it2,ids] = np.where( (U_up[:,:,:,:,it2,ids]<0.)&(U_low[:,:,:,:,it2,ids]>0.), dUdz[:,:,:,:,it2,ids] , dUdz_2_NaN[:,:,:,:,it2,ids] )

        #for ilev in levar:
        #    for it in timear:
        #        for ilat in latar:
        #            for ilon in lonar:
        #                dU2V2[ilev,it,ilat,ilon,it2,ids] = math.sqrt( ( dU[ilev,it,ilat,ilon,it2,ids]**2 ) + ( dV[ilev,it,ilat,ilon,it2,ids]**2 ) )

        #dU2V2dz[:,:,:,:,it2,ids] = dU2V2[:,:,:,:,it2,ids] / dz_sh[:,:,:,:,it2,ids]


        #Avg over it dim (avg all timesteps within 2wk slice) -> 2wk avg
        U_wind_sh_2wk[:,:,:,it2,ids] = np.mean( U_wind_sh[:,:,:,:,it2,ids] , axis=1 )   
        #V_wind_sh_2wk[:,:,:,it2,ids] = np.mean( V_wind_sh[:,:,:,:,it2,ids] , axis=1 )  
        dUdz_2wk[:,:,:,it2,ids] = np.mean( dUdz[:,:,:,:,it2,ids] , axis=1 )   

        loop_end_time = time.process_time()
        loop_time_taken = loop_end_time - loop_start_time
        loop_time_taken_min = loop_time_taken / 60.
        print('Loop 3 Time = ' + str(loop_time_taken_min) + ' mins' ) 

        #===================================================================================================
        
     
    
    #--- Out of it2 loop (2wk time periods), still in ds loop (month) ---
    ###__Winds__###
    #avg over it2 dim -> monthly avg
    loop_start_time = time.process_time()
    #if winds_on == 1:
    U_wind_sh_mth[:,:,:,ids] = np.mean( U_wind_sh_2wk[:,:,:,:,ids], axis=3 )
    #V_wind_sh_mth[:,:,:,ids] = np.mean( V_wind_sh_2wk[:,:,:,:,ids], axis=3 )
    dUdz_mth[:,:,:,ids] = np.mean( dUdz_2wk[:,:,:,:,ids], axis=3 )
    loop_end_time = time.process_time()
    loop_time_taken = loop_end_time - loop_start_time
    loop_time_taken_min = loop_time_taken / 60.
    print('Loop 4 Time = ' + str(loop_time_taken_min) + ' mins' ) 
    
    
#--- Out of it2 loop (2wk time periods) AND ds loop (month) ---
loop_start_time = time.process_time()
###__Winds__###
#avg over ids dim -> dataset avg

#if winds_on == 1:
U_wind_sh_ds[:,:,:] = np.mean( U_wind_sh_mth[:,:,:,:], axis=3 )
#V_wind_sh_ds[:,:,:] = np.mean( V_wind_sh_mth[:,:,:,:], axis=3 )
dUdz_ds[:,:,:] = np.mean( dUdz_mth[:,:,:,:], axis=3 )

loop_end_time = time.process_time()
loop_time_taken = loop_end_time - loop_start_time
loop_time_taken_min = loop_time_taken / 60.
print('Loop 5 Time = ' + str(loop_time_taken_min) + ' mins' ) 
    
#===================================================================================================
end_time = time.process_time()
time_taken = end_time - start_time
time_taken_min = time_taken / 60.
print('========================================')
print('Calculation Time = ' + str(time_taken_min) + ' mins' )  #3mths ~11mins


In [ ]:
#Removed from main loop as not using atm

#Adding in additional criteria (U +ve above and -ve below) : dUdz variable
# dUdz calculated using dz calculated from geopH

# Took too long (~6 mins) for all lats, so calculate for lat_plot_idx only

#(lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape),

dUdz_2 = np.ndarray((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dUdz_2_NaN = np.empty((lev_shape, time_shape, lat_shape, lon_shape, time_it_shape, ds_months_shape), dtype = float)
dUdz_2_NaN[:] = np.NaN

start_time = time.process_time()

#for it in timear:
#    for ilon in lonar:
#        for ilat in latar:
#            for ilev in levar:
#                dUdz_2[ilev,it,ilat,ilon, time_it_shape, ds_months_shape] = np.where( (U_up[ilev,it,ilat,ilon, time_it_shape, ds_months_shape]<0.)&(U_low[ilev,it,ilat,ilon, time_it_shape, ds_months_shape]>0.), dUdz[ilev,it,ilat,ilon, time_it_shape, ds_months_shape] , dUdz_2_NaN[ilev,it,ilat,ilon, time_it_shape, ds_months_shape] )

dUdz_2 = np.where( (U_up<0.)&(U_low>0.), dUdz , dUdz_2_NaN )
                        
end_time = time.process_time()
print(end_time - start_time) #357.5188 for all lats, 4.600 for one lat


In [3]:
#U_wind_sh_plot & SpE_sh at individual time step
#U_winds plotted above/at same alt as/below the SpE layer alt

#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    +50.2
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 
print('      Plot Lat = ' + lat_plot_str )
#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------


# Set chosen/multiple levs to plot for lat-LT at a specific alt
lev_sl_idx_plot_multi = np.array([4,8,14],dtype=int)  #[ 4,  8, 14] ~114km,~105km,95km
lev_sl_idx_plot_multi_shape = len(lev_sl_idx_plot_multi)  #3
lev_plot_ar = np.arange(0,lev_sl_idx_plot_multi_shape)   #[0,1,2]



time_plot_idx = [18]
timearrr = np.arange(0,len(time_plot_idx), dtype=int)


it2_plot = 1
str_it2_plot = str(it2_plot)

Monthfolderstr = 'Jun-Aug'
ds_months = ['07']
ids_plot = 0
ds_months_ar = [0] #np.arange(0,len(ids_plot), dtype=int)

str_ids_plot = str(ids_plot)






for ids in ds_months_ar:   #iterate over month
    
    start_cftime_date = '0001-' + str(ds_months[ids]) + '-01'
    times = xr.cftime_range(start=start_cftime_date, periods=672, freq="1H", calendar="noleap")    
    
   
    
    #--------------Set value for ids for plot variables-----------------------
    ids_plot = ids
    str_ids_plot = str(ids_plot)
    print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )
  

    for iplev in lev_plot_ar:   #iterate over multiple levels for lat-LT plots at a specific alt
        #-----------------------------------Lat-LT plot setup ----------------------------------------
        #print('Lat-LT plot setup:')
        lev_sl_idx_plot = lev_sl_idx_plot_multi[iplev]    #14 4.9e-04hPa~95km  #8 1.1e-04hPa~105km  #4 4.0e-05 hPa~114km

        lev_sl_plot = lev_sl[lev_sl_idx_plot]
        lev_sl_plot_str = str("%.1e" % lev_sl_plot) + ' hPa'
        print('      Plot Lev = ' + lev_sl_plot_str )
        
        Zavg_sl_plot = Zavg_sl[lev_sl_idx_plot]
        Zavg_sl_plot_str = str("%.0f" % Zavg_sl_plot) + ' km'
        print('      Approx Z = ' + Zavg_sl_plot_str )


        
        lev_sl_plot1 = lev_sl[lev_sl_idx_plot+1]
        lev_sl_plot1_str = str("%.1e" % lev_sl_plot1) + ' hPa'
        lev_sl_plot2 = lev_sl[lev_sl_idx_plot-1]
        lev_sl_plot2_str = str("%.1e" % lev_sl_plot2) + ' hPa'
        #--
        Zavg_sl_plot1 = Zavg_sl[lev_sl_idx_plot+1]
        Zavg_sl_plot1_str = str("%.0f" % Zavg_sl_plot1) + ' km'
        Zavg_sl_plot2 = Zavg_sl[lev_sl_idx_plot-1]
        Zavg_sl_plot2_str = str("%.0f" % Zavg_sl_plot2) + ' km'
        
        

        for itr in timearrr:  #
            if it2_plot == 0:
                plottime = times[ time_plot_idx[itr] ]
                #print('1= ' + str(plottime))
            else:
                plottime = times[ time_plot_idx[itr] + 336]
                #print('2= ' + str(plottime))

            plottimestr = str(plottime)
            #print('3= ' + plottimestr)
            
            print('Plot slice ' + str_it2_plot + ': ' + str(times[it2_plot]) )

            #----------------------------------U_wind_sh at specified height-----------------------------------------------------------
            #---------------------------------------------------------------------------------------------

            U_wind_sh_plot = U_wind_sh[lev_sl_idx_plot+1, itr , :, :,  it2_plot , ids_plot ]    #(25, 336, 96, 144, 2, 3)
            
            SpEs_plot = SpEs[lev_sl_idx_plot, itr , :, :,  it2_plot , ids_plot ]    #(25, 336, 96, 144, 2, 3) 
            

            #ticks = np.linspace(0, 12, 25)
            ticks = np.linspace(-120, 120, 7)
            #ticks = np.linspace(0, 40, 21) #(F)

            fig, ax = plt.subplots(figsize=(11, 7))

            C = ax.contour(timeconv_plot, lat, U_wind_sh_plot, 
                             levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             #cmap=jet)
                              colors='black')

                            
            #P = ax.quiver(timeconv_plot[::6], lat[::6], U_wind_sh_plot[::6,::6], 0 #V_wind_sh_ds_plot[::6,::6]
            #              , scale=100
            #              , scale_units='inches'
            #              , units='inches'
            #              , width=0.02
            #                ) 
            
            ticks2 = np.geomspace(1*10**2, 4*10**4, 30)
            P = ax.contourf(timeconv_plot, lat, SpEs_plot, 
                             levels=ticks2,
                             locator=ticker.LogLocator(),
                             #extend='min',
                             #colors='black'
                             cmap=jet
                             )
            

            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title('U_wind_sh_plot, Lat vs LT, ' + lev_sl_plot1_str + ' (approx '+ Zavg_sl_plot1_str + ')' 
                         + ' \n SpEs_plot, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ')' 
                         + ' \n UT ' + plottimestr )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)
            
            CB = fig.colorbar(P, fraction=0.05,format=ticker.FuncFormatter(fmt), pad=0.01)
            CB.set_label(r'Fe$^{+}$ density, cm$^{-3}$')
            
            cbar = fig.colorbar(C, pad=0.04)#, orientation='horizontal')#, fraction=0.025,ticks=ticks[::2],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            cbar.set_label(r'U, ms$^{-1}$')
            
            figname= 'U_wind_sh_plot: Lat vs LT, ' + lev_sl_plot1_str + ' & SpEs_plot ' + lev_sl_plot_str + ' ' + plottimestr + '.png'

            plt.tight_layout()
            plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Winds' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------


            
            
                       
            #----------------------------------U_wind_sh at specified height-----------------------------------------------------------
            #---------------------------------------------------------------------------------------------

            U_wind_sh_plot = U_wind_sh[lev_sl_idx_plot, itr , :, :,  it2_plot , ids_plot ]    #(25, 336, 96, 144, 2, 3)


            #ticks = np.linspace(0, 12, 25)
            ticks = np.linspace(-120, 120, 33)
            #ticks = np.linspace(0, 40, 21) #(F)

            #ticks = np.linspace(0, 12, 25)
            ticks = np.linspace(-120, 120, 7)
            #ticks = np.linspace(0, 40, 21) #(F)

            fig, ax = plt.subplots(figsize=(11, 7))

            C = ax.contour(timeconv_plot, lat, U_wind_sh_plot, 
                             levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             #cmap=jet)
                              colors='black')

                            
            #P = ax.quiver(timeconv_plot[::6], lat[::6], U_wind_sh_plot[::6,::6], 0 #V_wind_sh_ds_plot[::6,::6]
            #              , scale=100
            #              , scale_units='inches'
            #              , units='inches'
            #              , width=0.02
            #                ) 
            
            ticks2 = np.geomspace(1*10**2, 4*10**4, 30)
            P = ax.contourf(timeconv_plot, lat, SpEs_plot, 
                             levels=ticks2,
                             locator=ticker.LogLocator(),
                             #extend='min',
                             #colors='black'
                             cmap=jet
                             )
    
            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            #ax.set_title('U_wind_sh_plot, Lat vs LT, ' + lev_sl_plot2_str + ' (approx '+ Zavg_sl_plot2_str + ') \n UT ' + plottimestr )
            
            ax.set_title('U_wind_sh_plot, Lat vs LT, ' #+ lev_sl_plot2_str + ' (approx '+ Zavg_sl_plot2_str + ')' 
             + '& SpEs_plot, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ')' 
             + ' \n UT ' + plottimestr )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            CB = fig.colorbar(P, fraction=0.05,format=ticker.FuncFormatter(fmt), pad=0.01)
            CB.set_label(r'Fe$^{+}$ density, cm$^{-3}$')
            
            cbar = fig.colorbar(C, pad=0.04)#, orientation='horizontal')#, fraction=0.025,ticks=ticks[::2],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            cbar.set_label(r'U, ms$^{-1}$')

            figname= 'U_wind_sh_plot: Lat vs LT, ' + lev_sl_plot_str + ' & SpEs_plot ' + lev_sl_plot_str + ' ' + plottimestr + '.png'

            plt.tight_layout()
            plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Winds' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------

            
            
            
            
            
            #----------------------------------U_wind_sh at specified height-----------------------------------------------------------
            #---------------------------------------------------------------------------------------------

            U_wind_sh_plot = U_wind_sh[lev_sl_idx_plot-1, itr , :, :,  it2_plot , ids_plot ]    #(25, 336, 96, 144, 2, 3)


            #ticks = np.linspace(0, 12, 25)
            ticks = np.linspace(-120, 120, 33)
            #ticks = np.linspace(0, 40, 21) #(F)

            #ticks = np.linspace(0, 12, 25)
            ticks = np.linspace(-120, 120, 7)
            #ticks = np.linspace(0, 40, 21) #(F)

            fig, ax = plt.subplots(figsize=(11, 7))

            C = ax.contour(timeconv_plot, lat, U_wind_sh_plot, 
                             levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             #cmap=jet)
                              colors='black')

                            
            #P = ax.quiver(timeconv_plot[::6], lat[::6], U_wind_sh_plot[::6,::6], 0 #V_wind_sh_ds_plot[::6,::6]
            #              , scale=100
            #              , scale_units='inches'
            #              , units='inches'
            #              , width=0.02
            #                ) 
            
            ticks2 = np.geomspace(1*10**2, 4*10**4, 30)
            P = ax.contourf(timeconv_plot, lat, SpEs_plot, 
                             levels=ticks2,
                             locator=ticker.LogLocator(),
                             #extend='min',
                             #colors='black'
                             cmap=jet
                             )
    
            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            #ax.set_title('U_wind_sh_plot, Lat vs LT, ' + lev_sl_plot2_str + ' (approx '+ Zavg_sl_plot2_str + ') \n UT ' + plottimestr )
            
            ax.set_title('U_wind_sh_plot, Lat vs LT, ' + lev_sl_plot2_str + ' (approx '+ Zavg_sl_plot2_str + ')' 
             + ' \n SpEs_plot, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ')' 
             + ' \n UT ' + plottimestr )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            CB = fig.colorbar(P, fraction=0.05,format=ticker.FuncFormatter(fmt), pad=0.01)
            CB.set_label(r'Fe$^{+}$ density, cm$^{-3}$')
            
            cbar = fig.colorbar(C, pad=0.04)#, orientation='horizontal')#, fraction=0.025,ticks=ticks[::2],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            cbar.set_label(r'U, ms$^{-1}$')


            figname= 'U_wind_sh_plot: Lat vs LT, ' + lev_sl_plot2_str + ' & SpEs_plot ' + lev_sl_plot_str + ' ' + plottimestr + '.png'

            plt.tight_layout()
            plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Winds' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------

            
            
            
            

            #----------------------------------SpEs at specified height-----------------------------------------------------------
            #---------------------------------------------------------------------------------------------

#            SpEs_plot = SpEs[lev_sl_idx_plot, itr , :, :,  it2_plot , ids_plot ]    #(25, 336, 96, 144, 2, 3)     

#            ticks = np.geomspace(1*10**2, 2*10**4, 30)

            #ticks = np.linspace(0, 12, 25)
            #ticks = np.linspace(-120, 120, 33)
            #ticks = np.linspace(0, 40, 21) #(F)

#            fig, ax = plt.subplots(figsize=(8, 5))

#            C = ax.contourf(timeconv_plot, lat, SpEs_plot, 
#                             levels=ticks,
#                             locator=ticker.LogLocator(),
#                             #extend='min',
#                             cmap=jet)

            #P = ax.quiver(timeconv_plot[::6], lat[::6], U_wind_sh_plot[::6,::6], 0 #V_wind_sh_ds_plot[::6,::6]
            #              , scale=100
            #              , scale_units='inches'
            #              , units='inches'
            #              , width=0.02
            #                ) 

#            ax.set_ylabel('Latitude')
#            ax.set_yticks(np.linspace(-90,90, 7)) 
#            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

#            ax.set_xlabel('Local Time / hrs')
#            ax.set_xticks(np.linspace(0,24,13)) 
#            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

#            ax.set_title('SpEs_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + plottimestr )

#            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
#            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
#            def fmt(x, pos):
#                a, b = '{:.1e}'.format(x).split('e')
#                b = int(b)
#                return r'${} \times 10^{{{}}}$'.format(a, b)

#            cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

#            cbar.set_label(r'ms$^{-1}$')


#            figname= 'SpEs_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + plottimestr + '.png'

#            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/' + Monthfolderstr +'/Winds' +  '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------

           

      Plot Lat = 50.2
      Plot Lev = 6.0e-03 hPa : 1.5e-05 hPa
      Approx Z = 80 km : 129 km
Plot month 0: 07
      Plot Lev = 4.0e-05 hPa
      Approx Z = 114 km
Plot slice 1: 0001-07-01 01:00:00


NameError: name 'U_wind_sh' is not defined

In [4]:
#----WINDS PLOTS - AVGs ----#


time_plot_idx = [0]#,6,12,18]
timearrr = np.arange(0,len(time_plot_idx), dtype=int)


#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    +50.2
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 
print('      Plot Lat = ' + lat_plot_str )
#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------


# Set chosen/multiple levs to plot for lat-LT at a specific alt
lev_sl_idx_plot_multi = np.array([4],dtype=int)  #[ 4,  8, 14] ~114km,~105km,95km
lev_sl_idx_plot_multi_shape = len(lev_sl_idx_plot_multi)  #3
lev_plot_ar = np.arange(0,lev_sl_idx_plot_multi_shape)   #[0,1,2]






for iplev in lev_plot_ar:   #iterate over multiple levels for lat-LT plots at a specific alt
    #-----------------------------------Lat-LT plot setup ----------------------------------------
    #print('Lat-LT plot setup:')
    lev_sl_idx_plot = lev_sl_idx_plot_multi[iplev]    #14 4.9e-04hPa~95km  #8 1.1e-04hPa~105km  #4 4.0e-05 hPa~114km

    lev_sl_plot = lev_sl[lev_sl_idx_plot]
    lev_sl_plot_str = str("%.1e" % lev_sl_plot) + ' hPa'
    #print('      Plot Lev = ' + lev_sl_plot_str )

    Zavg_sl_plot = Zavg_sl[lev_sl_idx_plot]
    Zavg_sl_plot_str = str("%.0f" % Zavg_sl_plot) + ' km'
    #print('      Approx Z = ' + Zavg_sl_plot_str )
    #===================================================================================================================================================
    
    if ds_months_shape>1:
        #-------------------------------dUdz_ds, dataset average at specified height--------------------------------------------------------------
        #---------------------------------------------------------------------------------------------
        dUdz_ds_plot = dUdz_ds[lev_sl_idx_plot,:,:] 
        SpEs_Occ_Fr_b_dsavg_plot = SpEs_Occ_Fr_b_dsavg[lev_sl_idx_plot,:,:]
        #-----------------------------------------------------

        ticks = np.linspace(-0.004, 0.004, 17)
        ticks2 = np.linspace(0, 14, 15)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contour(timeconv_plot, lat, dUdz_ds_plot, 
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                       colors='black',
                         #cmap=jet
                      )

        ax.clabel(C, levels=ticks)

        D = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_dsavg_plot, 
                         levels=ticks2,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet
                      )


        #P = ax.quiver(timeconv_plot[::6], lat[::6], dUdz_ds_plot[::6,::6], 0
        #              #, scale=0.03
        #              #, scale_units='inches'
        #              #, units='inches'
        #              #, width=0.02
        #                )

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        if ds_months_shape>1:
            ax.set_title('SpEs_Occ_Fr_b_dsavg_plot & dUdz_ds_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )
        else:
            ax.set_title('SpEs_Occ_Fr_b_dsavg_plot & dUdz_ds_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07, ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        CB = fig.colorbar(D, orientation='horizontal')

        #-------------------

        cbar.set_label(r's$^{-1}$')
        if ds_months_shape>1:
            figname= 'SpEs_Occ_Fr_b_dsavg_plot & dUdz_ds_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
        else:
            figname= 'SpEs_Occ_Fr_b_dsavg_plot & dUdz_ds_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'


        plt.tight_layout()
        plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/'  + Monthfolderstr + '/Winds' + '/Lat-LT/'+str(figname), dpi = 400) #
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/'  + 'Jun-Aug' + '/Winds' + '/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------


        #----------------------------------U_wind_sh_ds, dataset average at specified height-----------------------------------------------------------
        #---------------------------------------------------------------------------------------------
        U_wind_sh_ds_plot = U_wind_sh_ds[lev_sl_idx_plot,:,:] 
        #V_wind_sh_ds_plot = V_wind_sh_ds[lev_sl_idx_plot,:,:]
        #-----------------------------------------------------
        #ticks = np.linspace(0, 12, 25)
        ticks = np.linspace(-80, 80, 33)
        #ticks = np.linspace(0, 40, 21) #(F)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot, lat, U_wind_sh_ds_plot, 
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        P = ax.quiver(timeconv_plot[::6], lat[::6], U_wind_sh_ds_plot[::6,::6], 0 #V_wind_sh_ds_plot[::6,::6]
                      , scale=100
                      , scale_units='inches'
                      , units='inches'
                      , width=0.02
                        ) 

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        if ds_months_shape>1:
            ax.set_title('U_wind_sh_ds_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )
        else:
            ax.set_title('U_wind_sh_ds_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'ms$^{-1}$')

        if ds_months_shape>1:
            figname= 'U_wind_sh_ds_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
        else:
            figname= 'U_wind_sh_ds_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'

        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/'  + Monthfolderstr + '/Winds' + '/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------

    
    
    
    
    
    
    for ids in ds_months_ar:   #iterate over month

        start_cftime_date = '0001-' + str(ds_months[ids]) + '-01'
        times = xr.cftime_range(start=start_cftime_date, periods=672, freq="1H", calendar="noleap")    



        #--------------Set value for ids for plot variables-----------------------
        ids_plot = ids
        str_ids_plot = str(ids_plot)
        #print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )

        
        #U_wind_sh_mth[lev_sl_idx_plot,:,:,ids_plot] 

        
        #-------------------------------dUdz_mth, monthly average at specified height--------------------------------------------------------------
        #---------------------------------------------------------------------------------------------
        dUdz_mth_plot =  dUdz_mth[lev_sl_idx_plot,:,:,ids_plot]
        SpEs_Occ_Fr_b_avg_plot = SpEs_Occ_Fr_b_avg[lev_sl_idx_plot,:,:,ids_plot]
        #-----------------------------------------------------

        ticks = np.linspace(-0.004, 0.004, 17)
        ticks2 = np.linspace(0, 14, 15)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contour(timeconv_plot, lat, dUdz_mth_plot, 
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                       colors='black',
                         #cmap=jet
                      )

        ax.clabel(C, levels=ticks)

        D = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_dsavg_plot, 
                         levels=ticks2,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet
                      )


        #P = ax.quiver(timeconv_plot[::6], lat[::6], dUdz_ds_plot[::6,::6], 0
        #              #, scale=0.03
        #              #, scale_units='inches'
        #              #, units='inches'
        #              #, width=0.02
        #                )

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        if ds_months_shape>1:
            ax.set_title('SpEs_Occ_Fr_b_avg_plot & dUdz_mth_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )
        else:
            ax.set_title('SpEs_Occ_Fr_b_avg_plot & dUdz_mth_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07, ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        CB = fig.colorbar(D, orientation='horizontal')

        #-------------------

        cbar.set_label(r's$^{-1}$')
        if ds_months_shape>1:
            figname= 'SpEs_Occ_Fr_b_avg_plot & dUdz_mth_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
        else:
            figname= 'SpEs_Occ_Fr_b_avg_plot & dUdz_mth_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'


        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/'  + Monthfolderstr + '/Winds' '/Lat-LT/'+str(figname), dpi = 400) #
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/' + str(Metal)+'p/Occurence_freq/Jun-Aug' + '/Winds'+ '/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------

        


        for it2 in time_ar_2wk:  #iterate over each 2wk sample period

            #--------------Set value for it2 for plot variables-----------------------
            it2_plot = it2
            str_it2_plot = str(it2_plot)




            for itr in timearrr:  #
                if it2_plot == 0:
                    plottime = times[ time_plot_idx[itr] ]
                    print(plottime)
                else:
                    plottime = times [ time_plot_idx[itr] + 336]
                    print(plottime)


                #------------------------ SpEs_plot -----------------------------
                #SpEs_plot = SpEs[:, itr , lat_plot_idx, :, it2_plot , ids_plot]


                #------------------------ Mptdens_sh[ilev,it,:,:,it2,ids] -----------------------------
                #Mptdens_sh_plot = Mptdens_sh[:, itr , lat_plot_idx, :, it2_plot , ids_plot]






      Plot Lat = 50.2
      Plot Lev = 6.0e-03 hPa : 1.5e-05 hPa
      Approx Z = 80 km : 129 km


NameError: name 'dUdz_ds' is not defined

In [ ]:
# Test plots for 5' avg global average variables:
    #Mptdens_avv1_b_5d
    #Mptdens_avv1b5d_l
    #max_Mptdens_avv1_b_5d
    #max_Mptdens_avv1b5d_l
#-----------------------------------------------------------


#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 

#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
#print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
#print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------


#levLTticks = np.linspace(0, 16, 17)

#-----------------------------------Lat-LT plot setup ----------------------------------------
# Set chosen/multiple levs to plot for lat-LT at a specific alt

lev_sl_idx_plot_multi = np.array([4],dtype=int)  #[ 4,  8, 14]
lev_sl_idx_plot_multi_shape = len(lev_sl_idx_plot_multi)  #3
lev_plot_ar = np.arange(0,lev_sl_idx_plot_multi_shape)   #[0,1,2]



for ids in ds_months_ar:   #iterate over month
    #--------------Set value for ids for plot variables-----------------------
    ids_plot = ids
    str_ids_plot = str(ids_plot)
    #print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )
    
    for it2 in time_ar_2wk:  #iterate over each 2wk sample period
            #--------------Set value for it2 for plot variables-----------------------
            it2_plot = it2
            str_it2_plot = str(it2_plot)
            #print('Plot time ' + str_it2_plot + ': ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] )
            #print('----------------------------------------')   
        
        
            #------------------------------------------------------------------
            #--------------------------------------------------------------------------------------------
            #---------------------------------------------------------------------

            Mptdens_avv1_b_5d_plot = Mptdens_avv1_b_5d[:,:,it2_plot,ids_plot]

            #-----------------------------------------------------
            ticks = np.geomspace(1*10**-1, 2*10**3, 30)


            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(intlat, Zavg_sl , Mptdens_avv1_b_5d_plot,   #lev_sl              
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             extend='min',
                             cmap=jet)

            ax.set_xlabel('Latitude')
            ax.set_xticks(np.linspace(-90,90, 7)) 
            ax.set_xticklabels((-90,-60,-30,0,30,60,90)) 
        
            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 
            
            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)

            ax.set_title(Crit_name +' '+ 'Mptdens_avv1_b_5d_plot, Lev vs Lat, ' + ' \n UT ' + times_str_min[ids_plot][it2_plot] ) 


            #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            #cbar.set_label(r'Density (cm$^{-3}$)')

            figname= Crit_name +' '+ 'Mptdens_avv1_b_5d_plot: Lat vs LT, ' + ' ' + times_str_min[ids_plot][it2_plot]  + '.png'


            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------


            #--------------------------------------------------------------------
            #---------------------------------------------------------------------------------------------
            #---------------------------------------------------------------------

            Mptdens_avv1b5d_l_plot = Mptdens_avv1b5d_l[:,:,it2_plot,ids_plot]

            #-----------------------------------------------------
            ticks = np.geomspace(1*10**-1, 2*10**3, 30)


            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(lat, Zavg_sl, Mptdens_avv1b5d_l_plot,       #lev_sl          
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             extend='min',
                             cmap=jet)

            ax.set_xlabel('Latitude')
            ax.set_xticks(np.linspace(-90,90, 7)) 
            ax.set_xticklabels((-90,-60,-30,0,30,60,90)) 
        
            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min)
            
            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
    
            ax.set_title(Crit_name +' '+ 'Mptdens_avv1b5d_l_plot, Lev vs Lat, ' + ' \n UT ' + times_str_min[ids_plot][it2_plot]  ) 

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            #cbar.set_label(r'Density (cm$^{-3}$)')

            figname= Crit_name +' '+ 'Mptdens_avv1b5d_l_plot: Lat vs LT, ' + ' ' + times_str_min[ids_plot][it2_plot]  + '.png'


            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------

            
            
            
            
            
            #------------------------------------------------------------------
            #--------------------------------------------------------------------------------------------
            #---------------------------------------------------------------------

            max_Mptdens_avv1_b_5d_plot = max_Mptdens_avv1_b_5d[:,it2_plot,ids_plot]

            #-----------------------------------------------------
            ticks = np.geomspace(1*10**-1, 2*10**3, 30)


            fig, ax = plt.subplots(figsize=(8, 5))

            
            C = ax.plot(intlat, max_Mptdens_avv1_b_5d_plot )
            
            ax.set_xlabel('Latitude')
            ax.set_xticks(np.linspace(-90,90, 7)) 
            ax.set_xticklabels((-90,-60,-30,0,30,60,90)) 


            ax.set_title(Crit_name +' '+ 'max_Mptdens_avv1_b_5d_plot, Lat dependence, ' + ' \n UT ' + times_str_min[ids_plot][it2_plot] ) 


            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)



            #cbar.set_label(r'Density (cm$^{-3}$)')

            figname= Crit_name +' '+ 'max_Mptdens_avv1_b_5d_plot: Lat, ' + ' ' + times_str_min[ids_plot][it2_plot]  + '.png'


            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------



            #------------------------------------------------------------------
            #--------------------------------------------------------------------------------------------
            #---------------------------------------------------------------------

            max_Mptdens_avv1b5d_l_plot = max_Mptdens_avv1b5d_l[:,it2_plot,ids_plot]

            #-----------------------------------------------------
            ticks = np.geomspace(1*10**-1, 2*10**3, 30)


            fig, ax = plt.subplots(figsize=(8, 5))

            
            C = ax.plot(lat, max_Mptdens_avv1b5d_l_plot )
            
            ax.set_xlabel('Latitude')
            ax.set_xticks(np.linspace(-90,90, 7)) 
            ax.set_xticklabels((-90,-60,-30,0,30,60,90)) 


            ax.set_title(Crit_name +' '+ 'max_Mptdens_avv1b5d_l_plot,  Lat dependence, ' + ' \n UT ' + times_str_min[ids_plot][it2_plot] ) 


            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)



            #cbar.set_label(r'Density (cm$^{-3}$)')

            figname= Crit_name +' '+ 'max_Mptdens_avv1b5d_l: Lat, ' + ' ' + times_str_min[ids_plot][it2_plot]  + '.png'


            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------




In [ ]:
#Metal density in half hour LT bins and 5' lat slices - monthly and dataset avg
    #Mptdens_avg_bb_5d_avg_plot
    #Mptdens_avg_bb_5d_dsavg_plot
    

#===================================================================================================
start_time = time.process_time() 
#===================================================================================================

# Set chosen/multiple lat slices to plot

intlat_plot_idx_multi = np.array([24],dtype=int)    #[20,22,24,26,28],dtype=int) #+ve lats 10-15', 20-25' --> 50-55'       #[7,9,11,13,15],dtype=int) #-ve lats 10-15', 20-25' --> 50-55'     
intlat_plot_idx_multi_shape = len(intlat_plot_idx_multi)  
intlat_plot_ar = np.arange(0,intlat_plot_idx_multi_shape) 



#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 

#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
#print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
#print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------


#levLTticks = np.linspace(0, 16, 17)



#-----------------------------------Lat-LT plot setup ----------------------------------------
# Set chosen/multiple levs to plot for lat-LT at a specific alt

lev_sl_idx_plot_multi = np.array([4],dtype=int)  #[ 4,  8, 14]
lev_sl_idx_plot_multi_shape = len(lev_sl_idx_plot_multi)  #3
lev_plot_ar = np.arange(0,lev_sl_idx_plot_multi_shape)   #[0,1,2]





for ids in ds_months_ar:   #iterate over month
    #--------------Set value for ids for plot variables-----------------------
    ids_plot = ids
    str_ids_plot = str(ids_plot)
    #print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )
      
        
        
    #-----------------------------------------------------
    #-----------------------global avg Fe+ layer ------------------------------
    #-----------------------------------------------------      
    Mptdens_avg_bb_5d_avg_plot_a = np.mean ( Mptdens_avg_bb_5d_avg[:,:,:,ids_plot], axis=2 ) #avg over lon
    #print(Mptdens_avg_bb_5d_avg_plot_a.shape)
    Mptdens_avg_bb_5d_avg_plot_avl = np.mean ( Mptdens_avg_bb_5d_avg_plot_a[:,:], axis=0 )  #avg over lev

    fig, ax = plt.subplots(figsize=(8, 5))
    C = ax.plot(Mptdens_avg_bb_5d_avg_plot_avl, intlat)
    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90))
    ax.set_title('Global avg '+ Metal + '+ layer density' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1])
    #-----------------------------------------------------  
        
        
    #===================================================================================================================================================
    for iplev in lev_plot_ar:   #iterate over multiple levels for lat-LT plots at a specific alt
        #-----------------------------------Lat-LT plot setup ----------------------------------------
        #print('Lat-LT plot setup:')
        lev_sl_idx_plot = lev_sl_idx_plot_multi[iplev]    #14 4.9e-04hPa~95km  #8 1.1e-04hPa~105km  #4 4.0e-05 hPa~114km

        lev_sl_plot = lev_sl[lev_sl_idx_plot]
        lev_sl_plot_str = str("%.1e" % lev_sl_plot) + ' hPa'
        #print('      Plot Lev = ' + lev_sl_plot_str )

        Zavg_sl_plot = Zavg_sl[lev_sl_idx_plot]
        Zavg_sl_plot_str = str("%.0f" % Zavg_sl_plot) + ' km'
        #print('      Approx Z = ' + Zavg_sl_plot_str )
        #===================================================================================================================================================

        
        
        
        #-----------------------------------------------------
        #---------------------------global avg Fe+ layer at plot lev --------------------------
        #-----------------------------------------------------
        Mptdens_avg_bb_5d_avg_plot_av = np.mean ( Mptdens_avg_bb_5d_avg[lev_sl_idx_plot,:,:,ids_plot], axis=1 )
        
        fig, ax = plt.subplots(figsize=(8, 5))
        C = ax.plot(Mptdens_avg_bb_5d_avg_plot_av, intlat)
        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 
        ax.set_title('Global avg '+ Metal + '+ layer density at ' + lev_sl_plot_str + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1])
        #-----------------------------------------------------
        
        
        

        
        
        
        #----------------------Monthly avg (lev, lat, LT, ids)----------------------------------------------
        #-----------------------------------Lat-LT----------------------------------------------------------
        #------------------------data averaged in 5' lat slices---------------------------------------------

        Mptdens_avg_bb_5d_avg_plot = Mptdens_avg_bb_5d_avg[lev_sl_idx_plot,:,:,ids_plot]
        
        #-----------------------------------------------------
        ticks = np.geomspace(1*10**-1, 2*10**3, 30)


        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot[::3], intlat, Mptdens_avg_bb_5d_avg_plot,                 
                         levels=ticks,
                         locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        if ds_months_shape>1:
            ax.set_title(Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] ) 
        else:
            ax.set_title(Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] ) 

        #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'Density (cm$^{-3}$)')

        if ds_months_shape>1:
            figname= Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot: Lat vs LT, ' + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
        else:
            figname= Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot: Lat vs LT, ' + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'

        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------

            
            
    for iintlatt in intlat_plot_ar:
        #print('Lev-LT plot setup (intlat):')
        intlat_plot_idx =  intlat_plot_idx_multi[iintlatt] 
        lat_plot_1 = intlat[intlat_plot_idx] -2.5
        lat_plot_2 = intlat[intlat_plot_idx] +2.5
        lat_plot_str1 = str("%.0f" % lat_plot_1) 
        lat_plot_str2 = str("%.0f" % lat_plot_2)
        #print('      Plot lat = ' + lat_plot_str1 + ':' + lat_plot_str2)
  
        
        
        #------------------------Monthly avg (lev, lat, LT, ids)--------------------------------------------
        #-----------------------------------Lev-LT----------------------------------------------------------
        #------------------------data averaged in 5' lat slices---------------------------------------------

        Mptdens_avg_bb_5d_avg_plot = Mptdens_avg_bb_5d_avg[:, intlat_plot_idx, :, ids_plot] 
        #-----------------------------------------------------

        ticks = np.geomspace(1*10**1, 5*10**3, 30)


        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot[::3], Zavg_sl, Mptdens_avg_bb_5d_avg_plot,  #lev_sl
                         levels=ticks,
                         locator=ticker.LogLocator(),
                         extend='min',
                         cmap=jet)

        #ax.set_yscale('log')
        #ax.set_ylabel('Pressure / HPa')
        #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 
        
        #ax2 = ax.twinx()
        #ax2.set_ylabel('Approx Geopotential Height / Km')
        #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
        #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])
        
        ax.set_ylabel('Approx Geopotential Height / Km')
        ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)

        
        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 
        
        if ds_months_shape>1:
            ax.set_title(Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot, Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ', \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] )
        else:
            ax.set_title(Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot, Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ', \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )           
            
        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        #ax.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'Density (cm$^{-3}$)')
        if ds_months_shape>1:
            figname= Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot: Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
        else:
            figname= Crit_name +' '+ 'Mptdens_avg_bb_5d_avg_plot: Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'

                         
        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/5deg_latbins'+ '/Lev-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------

        if ds_months_shape>1:
            #--------------------------Dataset avg (lev, lat, LT, ids)------------------------------------------
            #-----------------------------------Lev-LT----------------------------------------------------------
            #------------------------data averaged in 5' lat slices---------------------------------------------
            Mptdens_avg_bb_5d_dsavg_plot = Mptdens_avg_bb_5d_dsavg[:, intlat_plot_idx, :]

            #-----------------------------------------------------


            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot[::3], Zavg_sl, Mptdens_avg_bb_5d_dsavg_plot,    #lev_sl
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             extend='min',
                             cmap=jet)

            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

            #ax2 = ax.twinx()
            #ax2.set_ylabel('Approx Geopotential Height / Km')
            #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 
            ax.set_title(Crit_name +' '+ 'Mptdens_avg_bb_5d_dsavg_plot, Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ', \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            #ax.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            cbar.set_label(r'Density (cm$^{-3}$)')

            figname= Crit_name +' '+ 'Mptdens_avg_bb_5d_dsavg_plot: Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'


            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr+ '/5deg_latbins' + '/Lev-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------


        
        
        #for it2 in time_ar_2wk:  #iterate over each 2wk sample period
        #    #--------------Set value for it2 for plot variables-----------------------
        #    it2_plot = it2
        #    str_it2_plot = str(it2_plot)
        #    #print('Plot time ' + str_it2_plot + ': ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] )
        #    #print('----------------------------------------')   
   




        

                
#===================================================================================================
end_time = time.process_time()
time_taken = end_time - start_time
time_taken_min = time_taken / 60.
print('========================================')
print('Calculation Time = ' + str(time_taken_min) + ' mins' ) 

In [ ]:
#SpEs & Mptdens_sh


time_plot_idx = [0,6,12,18]
timearrr = np.arange(0,len(time_plot_idx), dtype=int)


#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 
print('      Plot Lat = ' + lat_plot_str )
#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------



for ids in ds_months_ar:   #iterate over month
    
    start_cftime_date = '0001-' + str(ds_months[ids]) + '-01'
    times = xr.cftime_range(start=start_cftime_date, periods=672, freq="1H", calendar="noleap")    
    
   
    
    #--------------Set value for ids for plot variables-----------------------
    ids_plot = ids
    str_ids_plot = str(ids_plot)
    print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )
       
    
    for it2 in time_ar_2wk:  #iterate over each 2wk sample period
        
        #--------------Set value for it2 for plot variables-----------------------
        it2_plot = it2
        str_it2_plot = str(it2_plot)
     
        
        for itr in timearrr:  #
            if it2_plot == 0:
                plottime = times[ time_plot_idx[itr] ]
            else:
                plottime = times [ time_plot_idx[itr] + 336]
                print(plottime)
                
                
    
            
            #------------------------ SpEs_plot -----------------------------
            SpEs_plot = SpEs[:, itr , lat_plot_idx, :, it2_plot , ids_plot]
            
            
            #ticks = np.geomspace(1*10**2, 1*10**3, 30)
            ticks = np.linspace(1*10**2, 2*10**4, 30)

            fig, ax = plt.subplots(figsize=(8, 5))

            #C = ax.pcolormesh(timeconv_plot, lev_sl, SpEs_plot)
            C = ax.contourf(timeconv_plot, Zavg_sl, SpEs_plot,    #lev_sl
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             extend='min',
                             cmap=jet)

            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

            #ax2 = ax.twinx()
            #ax2.set_ylabel('Approx Geopotential Height / Km')
            #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            
            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'SpEs_plot, Lev vs LT, lat' + lat_plot_str + ',\n UT ' + str(plottime) ) #+ str(times[itr]) )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::4])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            #cbar.set_label(r'Number of Occurences')

            figname= Crit_name +' '+ 'SpEs_plot: Lev vs LT, lat' + lat_plot_str + ' ' + str(plottime)  + '.png'
            plt.tight_layout()
            path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lev-LT/SpEs/'
            #plt.savefig(path+str(figname), dpi = 400) #
            
            
            
            
            
            
            
            #------------------------ Mptdens_sh[ilev,it,:,:,it2,ids] -----------------------------
            Mptdens_sh_plot = Mptdens_sh[:, itr , lat_plot_idx, :, it2_plot , ids_plot]
            
            
            #ticks = np.geomspace(1*10**2, 2*10**4, 30)

            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot, Zavg_sl, Mptdens_sh_plot,   #lev_sl
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             extend='min',
                             cmap=jet)

            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

            #ax2 = ax.twinx()
            #ax2.set_ylabel('Approx Geopotential Height / Km')
            #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            
            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'Mptdens_sh_plot, Lev vs LT, lat' + lat_plot_str + ',\n UT ' + str(plottime) ) #+ str(times[itr]) )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::4])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            #cbar.set_label(r'Number of Occurences')

            figname= Crit_name +' '+ 'Mptdens_sh_plot: Lev vs LT, lat' + lat_plot_str + ' ' + str(plottime)  + '.png'
            plt.tight_layout()
            path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lev-LT/SpEs/'
            #plt.savefig(path+str(figname), dpi = 400) #








In [ ]:
#Critical frequency plots:
    #SpEs_e            
    #edens_sh
    
    #maxSpEs__e
    #foEs_m
    #foEs_m_av
    #foEs_m_av_mth  
    #foEs_m_av_ds   
    


time_plot_idx = [12]#,6]#,12,18]
timearrr = np.arange(0,len(time_plot_idx), dtype=int)


#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 
print('      Plot Lat = ' + lat_plot_str )
#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------


#------------------------ foEs_m_av_ds -----------------------------
#------------------------ Lat-LT -----------------------------

ticks = np.linspace(0, 5, 21)

fig, ax = plt.subplots(figsize=(8, 5))

C = ax.contourf(timeconv_plot, lat, foEs_m_av_ds,
                 levels=ticks,
                 #locator=ticker.LogLocator(),
                 #extend='min',
                 cmap=jet)

ax.set_ylabel('Latitude')
ax.set_yticks(np.linspace(-90,90, 7)) 
ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

ax.set_xlabel('Local Time / hrs')
ax.set_xticks(np.linspace(0,24,13)) 
ax.set_xticklabels(timeconv_plot_ticks[::2]) 

ax.set_title(Crit_name +' '+ 'foEs_m_av_ds, Lat vs LT ' + ',\n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] ) #+ str(times[itr]) )

ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

cbar = fig.colorbar(C, pad=0.12)#,ticks=ticks[::4])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
#-------------------

cbar.set_label(r'Critical Frequency / MHz')

figname= Crit_name +' '+ 'foEs_m_av_ds: Lat vs LT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1]  + '.png'
plt.tight_layout()
path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lat-LT/SpEs/'
plt.savefig(path+str(figname), dpi = 400) #
#-------------------------------------------------------------------




for ids in ds_months_ar:   #iterate over month
    
    start_cftime_date = '0001-' + str(ds_months[ids]) + '-01'
    times = xr.cftime_range(start=start_cftime_date, periods=672, freq="1H", calendar="noleap")    
    
   
    
    #--------------Set value for ids for plot variables-----------------------
    ids_plot = ids
    str_ids_plot = str(ids_plot)
    print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )
    

    
    #------------------------ foEs_m_av_mth_plot -----------------------------
    #------------------------ Lat-LT -----------------------------
    foEs_m_av_mth_plot = foEs_m_av_mth[:, : , ids_plot]

    ticks = np.linspace(0, 5, 21)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot, lat, foEs_m_av_mth_plot,
                     levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    ax.set_title(Crit_name +' '+ 'foEs_m_av_mth_plot, Lat vs LT ' + ',\n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] ) #+ str(times[itr]) )

    ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    cbar = fig.colorbar(C, pad=0.12)#,ticks=ticks[::4])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'Critical Frequency / MHz')

    figname= Crit_name +' '+ 'foEs_m_av_mth_plot: Lat vs LT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1]  + '.png'
    plt.tight_layout()
    path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lat-LT/SpEs/'
    plt.savefig(path+str(figname), dpi = 400) #
    #-------------------------------------------------------------------
        
        
    
    for it2 in time_ar_2wk:  #iterate over each 2wk sample period
        
        #--------------Set value for it2 for plot variables-----------------------
        it2_plot = it2
        str_it2_plot = str(it2_plot)
     


        #------------------------ foEs_m_av_plot -----------------------------
        #------------------------ Lat-LT -----------------------------
        foEs_m_av_plot = foEs_m_av[:, :, it2_plot , ids_plot]

        ticks = np.linspace(0, 5, 21)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot, lat, foEs_m_av_plot,
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        ax.set_title(Crit_name +' '+ 'foEs_m_av_plot, Lat vs LT ' + ',\n UT ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] ) #+ str(times[itr]) )

        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        cbar = fig.colorbar(C, pad=0.12)#,ticks=ticks[::4])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'Critical Frequency / MHz')

        figname= Crit_name +' '+ 'foEs_m_av_plot: Lat vs LT ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot]  + '.png'
        plt.tight_layout()
        path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lat-LT/SpEs/'
        plt.savefig(path+str(figname), dpi = 400) #
        #-------------------------------------------------------------------
        
        
        
        
        for itr in timearrr:  #iterate over chosen timesteps for plotting
            if it2_plot == 0:
                plottime = times[ time_plot_idx[itr] ]
            else:
                plottime = times [ time_plot_idx[itr] + 336]
                print(plottime)
                

            #------------------------ maxSpEs_e_plot -----------------------------
            #------------------------ Lat-LT -----------------------------
            maxSpEs__e_plot = maxSpEs__e[itr , :, :, it2_plot , ids_plot]
            
            ticks = np.geomspace(1*10**3, 2*10**5, 30)
            #ticks = np.geomspace(1*10**9, 1*10**12, 30)
            
            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot, lat, maxSpEs__e_plot,
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             #extend='min',
                             cmap=jet)

            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 
        
            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'maxSpEs__e_plot, Lat vs LT ' + ',\n UT ' + str(plottime) ) #+ str(times[itr]) )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
            
            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)
        
            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::4],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            cbar.set_label(r'cm$^{-3}$')

            figname= Crit_name +' '+ 'maxSpEs__e_plot: Lat vs LT ' + str(plottime)  + '.png'
            plt.tight_layout()
            path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lat-LT/SpEs/'
            plt.savefig(path+str(figname), dpi = 400) #
            #-------------------------------------------------------------------
            
            

            #------------------------ foEs_m_plot -----------------------------
            #------------------------ Lat-LT -----------------------------
            foEs_m_plot = foEs_m[itr , :, :, it2_plot , ids_plot]
            
            ticks = np.linspace(0, 5, 21)

            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot, lat, foEs_m_plot,
                             levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             cmap=jet)

            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 
        
            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'foEs_m_plot, Lat vs LT ' + ',\n UT ' + str(plottime) ) #+ str(times[itr]) )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            cbar = fig.colorbar(C, pad=0.12)#,ticks=ticks[::4])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            cbar.set_label(r'Critical Frequency / MHz')

            figname= Crit_name +' '+ 'foEs_m_plot: Lat vs LT ' + str(plottime)  + '.png'
            plt.tight_layout()
            path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lat-LT/SpEs/'
            plt.savefig(path+str(figname), dpi = 400) #
            #-------------------------------------------------------------------
        
        
        
        

            #------------------------ SpEs_e_plot -----------------------------
            SpEs_e_plot = SpEs_e[:, itr , lat_plot_idx, :, it2_plot , ids_plot]
            
            ticks = np.geomspace(1*10**2, 2*10**5, 30)
            #ticks = np.linspace(1*10**2, 2*10**4, 30)

            fig, ax = plt.subplots(figsize=(8, 5))

            #C = ax.pcolormesh(timeconv_plot, lev_sl, SpEs_plot)
            C = ax.contourf(timeconv_plot, Zavg_sl, SpEs_e_plot,   #lev_sl
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             #extend='min',
                             cmap=jet)

            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

            #ax2 = ax.twinx()
            #ax2.set_ylabel('Approx Geopotential Height / Km')
            #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            
            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'SpEs_e_plot, Lev vs LT, lat' + lat_plot_str + ',\n UT ' + str(plottime) ) #+ str(times[itr]) )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
    
            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::4],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            #cbar.set_label(r'Number of Occurences')

            figname= Crit_name +' '+ 'SpEs_e_plot: Lev vs LT, lat' + lat_plot_str + ' ' + str(plottime)  + '.png'
            plt.tight_layout()
            path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lev-LT/SpEs/'
            plt.savefig(path+str(figname), dpi = 400) #
            #-------------------------------------------------------------------
            
            
            
            
            
            #------------------------ edens_sh[ilev,it,:,:,it2,ids] -----------------------------
            edens_sh_plot = edens_sh[:, itr , lat_plot_idx, :, it2_plot , ids_plot]
            
            
            ticks = np.geomspace(1*10**2, 2*10**5, 30)
            #ticks = np.linspace(1*10**2, 2*10**4, 30)

            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot, Zavg_sl, edens_sh_plot,   #lev_sl
                             levels=ticks,
                             locator=ticker.LogLocator(),
                             extend='min',
                             cmap=jet)

            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

            #ax2 = ax.twinx()
            #ax2.set_ylabel('Approx Geopotential Height / Km')
            #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            
            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'edens_sh_plot, Lev vs LT, lat' + lat_plot_str + ',\n UT ' + str(plottime) ) #+ str(times[itr]) )

            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::4],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            #cbar.set_label(r'Number of Occurences')

            figname= Crit_name +' '+ 'edens_sh_plot: Lev vs LT, lat' + lat_plot_str + ' ' + str(plottime)  + '.png'
            plt.tight_layout()
            path = './Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Lev-LT/SpEs/'
            plt.savefig(path+str(figname), dpi = 400) #
            #-------------------------------------------------------------------




In [ ]:
SpEs_Occ_Fr_b_dsavglevLT = np.mean(SpEs_Occ_Fr_b_dsavglev, axis=1)
SpEs_Occ_Fr_b_dsavglevLT.shape

In [ ]:
#SpEs Occ Freq plots - all

#===================================================================================================
start_time = time.process_time() 
#===================================================================================================

# Set chosen/multiple lat slices to plot

intlat_plot_idx_multi = np.array([20,22,24,26,28],dtype=int) #+ve lats 10-15', 20-25' --> 50-55'       #[7,9,11,13,15],dtype=int) #-ve lats 10-15', 20-25' --> 50-55'     
intlat_plot_idx_multi_shape = len(intlat_plot_idx_multi)  
intlat_plot_ar = np.arange(0,intlat_plot_idx_multi_shape) 


# Set chosen/multiple levs to plot for lat-LT at a specific alt

lev_sl_idx_plot_multi = np.array([4,8,14],dtype=int)  #[ 4,  8, 14]
lev_sl_idx_plot_multi_shape = len(lev_sl_idx_plot_multi)  #3
lev_plot_ar = np.arange(0,lev_sl_idx_plot_multi_shape)   #[0,1,2]


#-----------------------------------Lev-LT plot setup ----------------------------------------
lat_plot_idx = 74   #21=-50.2   #37 #=-19.9    
lat_plot = lat[lat_plot_idx]
lat_plot_str = str("%.1f" % lat_plot) 

#-----------------------------------
lev_sl_idx_plot_min = 0
lev_sl_idx_plot_max = 24
lev_sl_plot_min = lev_sl[lev_sl_idx_plot_min]
lev_sl_plot_max = lev_sl[lev_sl_idx_plot_max]
lev_sl_plot_min_str = str("%.1e" % lev_sl_plot_min) + ' hPa'
lev_sl_plot_max_str = str("%.1e" % lev_sl_plot_max) + ' hPa'
#print('      Plot Lev = ' + lev_sl_plot_max_str + ' : ' + lev_sl_plot_min_str)

Zavg_sl_plot_min = Zavg_sl[lev_sl_idx_plot_min]
Zavg_sl_plot_max = Zavg_sl[lev_sl_idx_plot_max]
Zavg_sl_plot_min_str = str("%.0f" % Zavg_sl_plot_min) + ' km'
Zavg_sl_plot_max_str = str("%.0f" % Zavg_sl_plot_max) + ' km'
#print('      Approx Z = ' + Zavg_sl_plot_max_str + ' : ' + Zavg_sl_plot_min_str)

Zavg_sl_plott = Zavg_sl[lev_sl_idx_plot_min:lev_sl_idx_plot_max+1]
Zavg_sl_plott = Zavg_sl_plott.to_numpy()
Zavg_sl_plottt = np.round_(Zavg_sl_plott, decimals=0)

#-------------------Set up Local Time Ticks-------------------------------
timeconv = lon / 15
timeconv_plot = timeconv.to_numpy() 
timeconv_plot_ticks = np.arange(0,25, dtype=float)

#------------Set string values for sigma symbol to use in plot------------
sigma = '\u03C3'
#----------------------------------------


levLTticks = np.linspace(0, 16, 17)

if ds_months_shape>1:
    #--------------------------Dataset avg (lev, lat, LT, ids)------------------------------------------
    #-----------------------------------Lev-LT----------------------------------------------------------
    #---------------------------------------------------------------------------------------------------

    SpEs_Occ_Fr_b_dsavg_plot = SpEs_Occ_Fr_b_dsavg[:,lat_plot_idx,:] 

    #-----------------------------------------------------
    #ticks = np.linspace(0, 14, 15)
    ticks = levLTticks
    #ticks = np.linspace(0, 40, 21) #(F)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot[::3], Zavg_sl, SpEs_Occ_Fr_b_dsavg_plot, #lev_sl
                     levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    #ax.set_yscale('log')
    #ax.set_ylabel('Pressure / HPa')
    #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

    #ax2 = ax.twinx()
    #ax2.set_ylabel('Approx Geopotential Height / Km')
    #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
    #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

    ax.set_ylabel('Approx Geopotential Height / Km')
    ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
    

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavg_plot, Lev vs LT, lat' + lat_plot_str + ', \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )

    ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    #ax.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
    plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    #------------------
    def fmt(x, pos):
        a, b = '{:.1e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'%')

    figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavg_plot: Lev vs LT, lat' + lat_plot_str + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lev-LT/'+str(figname), dpi = 400) #
    #---------------------------------------------------------------------------------------------------


    
    #--------------------Dataset avg over all heights (Lat)-----------------------------------------
    #---------------------------------------------------------------------------------------------
    
    
    ticks = np.linspace(0, 8, 17)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.plot(SpEs_Occ_Fr_b_dsavglevLT, lat)

    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

    ax.set_xlabel('SpE Occ Frequency / %')
    #ax.set_xticks(np.linspace(0,24,13)) 
    #ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavglevLT, ' + ' \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )

    #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavglevLT: Lat, ' + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
    plt.tight_layout()
    plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lat-LT/'+str(figname), dpi = 400) #

    #--------------------------------------------------------------------------------------------------- 
    
    
    
    
    

    #--------------------Dataset avg over all heights (lat, LT)-----------------------------------------
    #-----------------------------------Lat-LT----------------------------------------------------------
    #---------------------------------------------------------------------------------------------------


    ticks = np.linspace(0, 8, 17)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_dsavglev, 
                     levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavglev, Lat vs LT, ' + ' \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )

    #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    #------------------
    def fmt(x, pos):
        a, b = '{:.1e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'%')

    figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavglev: Lat vs LT, ' + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lat-LT/'+str(figname), dpi = 400) #

    #--------------------------------------------------------------------------------------------------- 
    
    
    

    #--------------------Dataset avg over all heights (lat, LT)-----------------------------------------
    #-----------------------------------Lat-LT----------------------------------------------------------
    #------------------------data averaged in 5' lat slices---------------------------------------------

    SpEs_Occ_Fr_bb_5d_dsavglev_plot = SpEs_Occ_Fr_bb_5d_dsavglev[:,:]  #Whole Dataset avg over all heights ->(36, 48) lat,LT

    #-----------------------------------------------------

    #ticks = np.linspace(0, 8, 17)
    #ticks = np.linspace(0, 40, 21) #(F)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot[::3], intlat, SpEs_Occ_Fr_bb_5d_dsavglev_plot,

                     #levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_dsavglev_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] ) 

    #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    #------------------
    def fmt(x, pos):
        a, b = '{:.1e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar = fig.colorbar(C, pad=0.07)#,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'%')

    figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_dsavglev_plot: Lat vs LT, ' + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'


    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr+ '/Criteria_5deglats/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
    #---------------------------------------------------------------------------------------------------




for ids in ds_months_ar:   #iterate over month
    #--------------Set value for ids for plot variables-----------------------
    ids_plot = ids
    str_ids_plot = str(ids_plot)
    #print('Plot month ' + str_ids_plot + ': ' + str(ds_months[ids_plot]) )
    

    
    #------------------------Monthly avg (lev, lat, LT, ids)--------------------------------------------
    #-----------------------------------Lev-LT----------------------------------------------------------
    #---------------------------------------------------------------------------------------------------

    SpEs_Occ_Fr_b_avg_plot = SpEs_Occ_Fr_b_avg[:,lat_plot_idx,:,ids_plot] 

    #-----------------------------------------------------
    #ticks = np.linspace(0, 14, 15)
    ticks = levLTticks
    #ticks = np.linspace(0, 16, 17)
    #ticks = np.linspace(0, 40, 21) #(F)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot[::3], Zavg_sl, SpEs_Occ_Fr_b_avg_plot, #lev_sl
                     levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    #ax.set_yscale('log')
    #ax.set_ylabel('Pressure / HPa')
    #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

    #ax2 = ax.twinx()
    #ax2.set_ylabel('Approx Geopotential Height / Km')
    #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
    #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])
    
    ax.set_ylabel('Approx Geopotential Height / Km')
    ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)


    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 
    
    if ds_months_shape>1:
        ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot, Lev vs LT, lat' + lat_plot_str + ', \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] )
    else:
        ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot, Lev vs LT, lat' + lat_plot_str + ', \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

    
    ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    #ax.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
    plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    #------------------
    def fmt(x, pos):
        a, b = '{:.1e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'%')

    if ds_months_shape>1:
        figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot: Lev vs LT, lat' + lat_plot_str + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
    else:
        figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot: Lev vs LT, lat' + lat_plot_str + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'
    
    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lev-LT/'+str(figname), dpi = 400) #
    #---------------------------------------------------------------------------------------------------
    
    
    
    
    #-------------------Monthly avg over all heights (lat, LT, ids)-------------------------------------
    #-----------------------------------Lat-LT----------------------------------------------------------
    #---------------------------------------------------------------------------------------------------

    SpEs_Occ_Fr_b_avglev_plot = SpEs_Occ_Fr_b_avglev[:,:,ids_plot]

    #-----------------------------------------------------

    ticks = np.linspace(0, 8, 17)
    #ticks = np.linspace(0, 40, 21) #(F)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_avglev_plot, 
                     levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    if ds_months_shape>1:
        ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_avglev_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] )
    else:
        ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_avglev_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )
        
    #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    #------------------
    def fmt(x, pos):
        a, b = '{:.1e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'%')

    if ds_months_shape>1:
        figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_avglev_plot: Lat vs LT, ' + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
    else:
        figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_avglev_plot: Lat vs LT, ' + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'
        
    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lat-LT/'+str(figname), dpi = 400) #
    #---------------------------------------------------------------------------------------------------

    
    
    #-------------------Monthly avg over all heights (lat, LT, ids)-------------------------------------
    #-----------------------------------Lat-LT----------------------------------------------------------
    #------------------------data averaged in 5' lat slices---------------------------------------------

    SpEs_Occ_Fr_bb_5d_avglev_plot = SpEs_Occ_Fr_bb_5d_avglev[:,:,ids_plot]  # =Monthly avg over all heights   ->(36, 48, 3)   lat, LT,month 

    #-----------------------------------------------------

    #ticks = np.linspace(0, 8, 17)
    #ticks = np.linspace(0, 40, 21) #(F)

    fig, ax = plt.subplots(figsize=(8, 5))

    C = ax.contourf(timeconv_plot[::3], intlat, SpEs_Occ_Fr_bb_5d_avglev_plot, 
                    #levels=ticks,
                     #locator=ticker.LogLocator(),
                     #extend='min',
                     cmap=jet)

    ax.set_ylabel('Latitude')
    ax.set_yticks(np.linspace(-90,90, 7)) 
    ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    if ds_months_shape>1:
        ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avglev_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] ) 
    else:
        ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avglev_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

    ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

    #------------------
    def fmt(x, pos):
        a, b = '{:.1e}'.format(x).split('e')
        b = int(b)
        return r'${} \times 10^{{{}}}$'.format(a, b)

    cbar = fig.colorbar(C, pad=0.07)#,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    cbar.set_label(r'%')
    
    if ds_months_shape>1:
        figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avglev_plot: Lat vs LT, ' + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
    else:
        figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avglev_plot: Lat vs LT, ' + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'

    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/5deg_latbins'+ '/Lat-LT/'+str(figname), dpi = 400) #
    #---------------------------------------------------------------------------------------------------
    
  
    
    for iplev in lev_plot_ar:   #iterate over multiple levels for lat-LT plots at a specific alt
        #-----------------------------------Lat-LT plot setup ----------------------------------------
        #print('Lat-LT plot setup:')
        lev_sl_idx_plot = lev_sl_idx_plot_multi[iplev]    #14 4.9e-04hPa~95km  #8 1.1e-04hPa~105km  #4 4.0e-05 hPa~114km

        lev_sl_plot = lev_sl[lev_sl_idx_plot]
        lev_sl_plot_str = str("%.1e" % lev_sl_plot) + ' hPa'
        #print('      Plot Lev = ' + lev_sl_plot_str )

        Zavg_sl_plot = Zavg_sl[lev_sl_idx_plot]
        Zavg_sl_plot_str = str("%.0f" % Zavg_sl_plot) + ' km'
        #print('      Approx Z = ' + Zavg_sl_plot_str )
        #===================================================================================================================================================


        #----------------------Monthly avg (lev, lat, LT, ids)----------------------------------------------
        #-----------------------------------Lat-LT----------------------------------------------------------
        #---------------------------------------------------------------------------------------------------

        SpEs_Occ_Fr_b_avg_plot = SpEs_Occ_Fr_b_avg[lev_sl_idx_plot,:,:,ids_plot] 

        #-----------------------------------------------------
        #ticks = np.linspace(0, 12, 25)
        ticks = np.linspace(0, 14, 15)
        #ticks = np.linspace(0, 40, 21) #(F)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_avg_plot, 
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 
        
        if ds_months_shape>1:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] )
        else:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

            
        #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'%')

        if ds_months_shape>1:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
        else:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_avg_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'
        
        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------

        
        #----------------------Monthly avg (lev, lat, LT, ids)----------------------------------------------
        #-----------------------------------Lat-LT----------------------------------------------------------
        #------------------------data averaged in 5' lat slices---------------------------------------------

        #Need to move plots for these two below
        SpEs_Occ_Fr_bb_5d_avg_plot = SpEs_Occ_Fr_bb_5d_avg[lev_sl_idx_plot,:,:,ids_plot]   # =Monthly avg   ->(25, 36, 48, 3)    lev, lat, LT, month
        #SpEs_Occ_Fr_bb_5d_dsavg_plot = SpEs_Occ_Fr_bb_5d_dsavg[lev_sl_idx_plot,:,:]    # =Whole Dataset avg   ->(25, 36, 48)  lev, lat, LT

        #-----------------------------------------------------

        #ticks = np.linspace(0, 8, 17)
        #ticks = np.linspace(0, 40, 21) #(F)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot[::3], intlat, SpEs_Occ_Fr_bb_5d_avg_plot,                 
                         #levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        if ds_months_shape>1:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] ) 
        else:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] ) 
        
        #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07)#,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'%')

        if ds_months_shape>1:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot: Lat vs LT, ' + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
        else:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot: Lat vs LT, ' + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'

        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/5deg_latbins' + '/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------
    
        if ds_months_shape>1:
            #--------------------------Dataset avg (lev, lat, LT)-----------------------------------------------
            #-----------------------------------Lat-LT----------------------------------------------------------
            #---------------------------------------------------------------------------------------------------

            SpEs_Occ_Fr_b_dsavg_plot = SpEs_Occ_Fr_b_dsavg[lev_sl_idx_plot,:,:] 

            #-----------------------------------------------------
            ticks = np.linspace(0, 14, 15)
            #ticks = np.linspace(0, 8, 26)
            #ticks = np.linspace(0, 40, 21) #(F)

            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_dsavg_plot, 
                             levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             cmap=jet)

            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavg_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )

            #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            cbar.set_label(r'%')

            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_dsavg_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'
            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------


            #--------------------------Dataset avg (lev, lat, LT)-----------------------------------------------
            #-----------------------------------Lat-LT----------------------------------------------------------
            #------------------------data averaged in 5' lat slices---------------------------------------------


            SpEs_Occ_Fr_bb_5d_dsavg_plot = SpEs_Occ_Fr_bb_5d_dsavg[lev_sl_idx_plot,:,:]    # =Whole Dataset avg   ->(25, 36, 48)  lev, lat, LT

            #-----------------------------------------------------

            #ticks = np.linspace(0, 8, 17)
            #ticks = np.linspace(0, 40, 21) #(F)

            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot[::3], intlat, SpEs_Occ_Fr_bb_5d_avg_plot,                 
                             #levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             cmap=jet)

            ax.set_ylabel('Latitude')
            ax.set_yticks(np.linspace(-90,90, 7)) 
            ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 

            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_dsavg_plot, Lat vs LT, ' + ' \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] ) 

            #ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            #plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            cbar = fig.colorbar(C, pad=0.07)#,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            cbar.set_label(r'%')

            figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_dsavg_plot: Lat vs LT, ' + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'

            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/5deg_latbins'+ '/Lat-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------
        


            
        for iintlatt in intlat_plot_ar:
            #print('Lev-LT plot setup (intlat):')
            intlat_plot_idx =  intlat_plot_idx_multi[iintlatt] 
            lat_plot_1 = intlat[intlat_plot_idx] -2.5
            lat_plot_2 = intlat[intlat_plot_idx] +2.5
            lat_plot_str1 = str("%.0f" % lat_plot_1) 
            lat_plot_str2 = str("%.0f" % lat_plot_2)
            #print('      Plot lat = ' + lat_plot_str1 + ':' + lat_plot_str2)


            #------------------------Monthly avg (lev, lat, LT, ids)--------------------------------------------
            #-----------------------------------Lev-LT----------------------------------------------------------
            #------------------------data averaged in 5' lat slices---------------------------------------------

            SpEs_Occ_Fr_bb_5d_avg_plot2 = SpEs_Occ_Fr_bb_5d_avg[:,intlat_plot_idx,:,ids_plot]   # =Monthly avg   ->(25, 36, 48, 3)    lev, lat, LT, month

            #-----------------------------------------------------

            #ticks = np.linspace(0, 14, 15)
            ticks = levLTticks
            #ticks = np.linspace(0, 16, 17)
            #ticks = np.linspace(0, 40, 21) #(F)

            fig, ax = plt.subplots(figsize=(8, 5))

            C = ax.contourf(timeconv_plot[::3], Zavg_sl, SpEs_Occ_Fr_bb_5d_avg_plot2,   #lev_sl
                             levels=ticks,
                             #locator=ticker.LogLocator(),
                             #extend='min',
                             cmap=jet)

            #ax.set_yscale('log')
            #ax.set_ylabel('Pressure / HPa')
            #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

            #ax2 = ax.twinx()
            #ax2.set_ylabel('Approx Geopotential Height / Km')
            #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

            ax.set_ylabel('Approx Geopotential Height / Km')
            ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
            

            ax.set_xlabel('Local Time / hrs')
            ax.set_xticks(np.linspace(0,24,13)) 
            ax.set_xticklabels(timeconv_plot_ticks[::2]) 
            
            if ds_months_shape>1:
                ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot2, Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ', \n UT ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] )
            else:
                ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot2, Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ', \n UT ' + times_str_min[0] + ' : ' + times_str_max[-1] )

                
            ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
            #ax.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
            plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

            #------------------
            def fmt(x, pos):
                a, b = '{:.1e}'.format(x).split('e')
                b = int(b)
                return r'${} \times 10^{{{}}}$'.format(a, b)

            cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
            #-------------------

            cbar.set_label(r'%')

            if ds_months_shape>1:
                figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot2: Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ' ' + times_str_min[ids_plot][0] + ' : ' + times_str_max[ids_plot][-1] + '.png'
            else:
                figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_avg_plot2: Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ' ' + times_str_min[0] + ' : ' + times_str_max[-1] + '.png'
 
            
            plt.tight_layout()
            #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/5deg_latbins'+ '/Lev-LT/'+str(figname), dpi = 400) #
            #---------------------------------------------------------------------------------------------------
            
            if ds_months_shape>1:
                #--------------------------Dataset avg (lev, lat, LT, ids)------------------------------------------
                #-----------------------------------Lev-LT----------------------------------------------------------
                #------------------------data averaged in 5' lat slices---------------------------------------------

                SpEs_Occ_Fr_bb_5d_dsavg_plot2 = SpEs_Occ_Fr_bb_5d_dsavg[:,intlat_plot_idx,:]    # =Whole Dataset avg   ->(25, 36, 48)  lev, lat, LT

                #-----------------------------------------------------

                #ticks = np.linspace(0, 14, 15)
                ticks = levLTticks
                #ticks = np.linspace(0, 16, 17)
                #ticks = np.linspace(0, 40, 21) #(F)

                fig, ax = plt.subplots(figsize=(8, 5))

                C = ax.contourf(timeconv_plot[::3], Zavg_sl, SpEs_Occ_Fr_bb_5d_dsavg_plot2,   #lev_sl
                                 levels=ticks,
                                 #locator=ticker.LogLocator(),
                                 #extend='min',
                                 cmap=jet)

                #ax.set_yscale('log')
                #ax.set_ylabel('Pressure / HPa')
                #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

                #ax2 = ax.twinx()
                #ax2.set_ylabel('Approx Geopotential Height / Km')
                #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
                #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

                ax.set_ylabel('Approx Geopotential Height / Km')
                ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
                

                ax.set_xlabel('Local Time / hrs')
                ax.set_xticks(np.linspace(0,24,13)) 
                ax.set_xticklabels(timeconv_plot_ticks[::2]) 

                ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_dsavg_plot2, Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ', \n UT ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] )

                ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
                #ax.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
                plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

                #------------------
                def fmt(x, pos):
                    a, b = '{:.1e}'.format(x).split('e')
                    b = int(b)
                    return r'${} \times 10^{{{}}}$'.format(a, b)

                cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
                #-------------------

                cbar.set_label(r'%')

                figname= Crit_name +' '+ 'SpEs_Occ_Fr_bb_5d_dsavg_plot2: Lev vs LT, lat'  + lat_plot_str1 + ':' + lat_plot_str2 + ' ' + times_str_min[0][0] + ' : ' + times_str_max[-1][-1] + '.png'


                plt.tight_layout()
                #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr+ '/Criteria_5deglats/5deg_latbins' + '/Lev-LT/'+str(figname), dpi = 400) #
                #---------------------------------------------------------------------------------------------------
            
            
                
                
    for it2 in time_ar_2wk:  #iterate over each 2wk sample period
        #--------------Set value for it2 for plot variables-----------------------
        it2_plot = it2
        str_it2_plot = str(it2_plot)
        #print('Plot time ' + str_it2_plot + ': ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] )
        #print('----------------------------------------')   
        
        
    
        #--------------------------------2 week period------------------------------------------------------
        #-----------------------------------Lat-LT----------------------------------------------------------
        #---------------------------------------------------------------------------------------------------

        SpEs_Occ_Fr_b_plot = SpEs_Occ_Fr_b[lev_sl_idx_plot,:,:,it2_plot,ids_plot]

        #-----------------------------------------------------
        ticks = np.linspace(0, 14, 15)
        #ticks = np.linspace(0, 20, 21)
        #ticks = np.linspace(0, 16, 17)
        #ticks = np.linspace(0, 40, 21) #(F)

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot[::3], lat, SpEs_Occ_Fr_b_plot, 
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        ax.set_ylabel('Latitude')
        ax.set_yticks(np.linspace(-90,90, 7)) 
        ax.set_yticklabels((-90,-60,-30,0,30,60,90)) 

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 
        
        if ds_months_shape>1:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] )
        else:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_plot, Lat vs LT, ' + lev_sl_plot_str + ' (approx '+ Zavg_sl_plot_str + ') \n UT ' + times_str_min[it2_plot] + ' : ' + times_str_max[it2_plot] )

            
        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.07,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'%')
        
        if ds_months_shape>1:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] + '.png'
        else:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_plot: Lat vs LT, ' + lev_sl_plot_str + ' ' + times_str_min[it2_plot] + ' : ' + times_str_max[it2_plot] + '.png'

        
        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lat-LT/'+str(figname), dpi = 400) #
        #---------------------------------------------------------------------------------------------------


        #-------------------------------2 week period ------------------------------------------------------
        #-----------------------------------Lev-LT----------------------------------------------------------
        #---------------------------------------------------------------------------------------------------

        SpEs_Occ_Fr_b_plot = SpEs_Occ_Fr_b[:,lat_plot_idx,:,it2_plot,ids_plot] 

        #------------------------ SpEs_Occ_Freq_plot -----------------------------

        #ticks = np.linspace(0, 16, 17)
        #ticks = np.linspace(0, 14, 15)
        ticks = levLTticks

        fig, ax = plt.subplots(figsize=(8, 5))

        C = ax.contourf(timeconv_plot[::3], Zavg_sl, SpEs_Occ_Fr_b_plot,    #lev_sl
                         levels=ticks,
                         #locator=ticker.LogLocator(),
                         #extend='min',
                         cmap=jet)

        #ax.set_yscale('log')
        #ax.set_ylabel('Pressure / HPa')
        #ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

        #ax2 = ax.twinx()
        #ax2.set_ylabel('Approx Geopotential Height / Km')
        #ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
        #ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])
        
        ax.set_ylabel('Approx Geopotential Height / Km')
        ax.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)

        ax.set_xlabel('Local Time / hrs')
        ax.set_xticks(np.linspace(0,24,13)) 
        ax.set_xticklabels(timeconv_plot_ticks[::2]) 

        if ds_months_shape>1:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_plot, Lev vs LT, lat' + lat_plot_str + ', \n UT ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] )
        else:
            ax.set_title(Crit_name +' '+ 'SpEs_Occ_Fr_b_plot, Lev vs LT, lat' + lat_plot_str + ', \n UT ' + times_str_min[it2_plot] + ' : ' + times_str_max[it2_plot] )

        
        ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
        plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)

        #------------------
        def fmt(x, pos):
            a, b = '{:.1e}'.format(x).split('e')
            b = int(b)
            return r'${} \times 10^{{{}}}$'.format(a, b)

        cbar = fig.colorbar(C, pad=0.12,ticks=ticks[::2])#,format=ticker.FuncFormatter(fmt))#fraction=0.023, 
        #-------------------

        cbar.set_label(r'%')

        if ds_months_shape>1:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_plot: Lev vs LT, lat' + lat_plot_str + ' ' + times_str_min[ids_plot][it2_plot] + ' : ' + times_str_max[ids_plot][it2_plot] + '.png'
        else:
            figname= Crit_name +' '+ 'SpEs_Occ_Fr_b_plot: Lev vs LT, lat' + lat_plot_str + ' ' + times_str_min[it2_plot] + ' : ' + times_str_max[it2_plot] + '.png'

        
        plt.tight_layout()
        #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/' + Monthfolderstr + '/Criteria_5deglats/Lev-LT/' +str(figname), dpi = 400) #

        #---------------------------------------------------------------------------------------------------
        



                
#===================================================================================================
end_time = time.process_time()
time_taken = end_time - start_time
time_taken_min = time_taken / 60.
print('========================================')
print('Calculation Time = ' + str(time_taken_min) + ' mins' )  #11.21 mins


In [ ]:
#---------------------------------------------------------------------------------------------------

In [ ]:
#Individual timesteps during 2 week period  

#---------------------------------------------
#Set number of plots to create
time_plot2_ntimesteps = 12
#---------------------------------------------




#---------------------------------------------------------------------------------------------------
#-----------------------------------Lev-LT----------------------------------------------------
#---------------------------------------------------------------------------------------------------

time_plot2_idx_start = time_ind_2wk_min[it2_plot]
time_plot2_idx_end = time_ind_2wk_min[it2_plot] + time_plot2_ntimesteps

time_plot2_ar = np.arange(time_plot2_idx_start,time_plot2_idx_end)  #e.g. 336,337,338 for it2_plot=1 n=3

time_arr = np.arange(0,time_plot2_ntimesteps)
#time_plot2_ar = np.arange(0,time_plot2_ntimesteps) 

#for itime in time_plot2_ar:
for itime in time_arr:
    time_plot2_idx = time_plot2_ar[itime]
    
    #--------------- Plot setup: time -------------------------
    time_plot2 = timee[time_plot2_idx]
    times_plot2 = times[time_plot2_idx]
    times_plot2_str = str(times_plot2)
    #print('Plot time = ' + times_plot2_str)   
    
    #---------------Set up plot variables----------------------------
    SpEs_freq_bool_plot2 = SpEs_freq_bool[:,itime,lat_plot_idx,:,it2_plot,ids_plot]
    SpEs_freq_plot2 = SpEs_freq[:,itime,lat_plot_idx,:,it2_plot,ids_plot]
    
    #------------------------ SpEs_sh_freq_time_plot -----------------------------

    #ticks = np.linspace(0, 40, 33)

    fig, ax = plt.subplots(figsize=(8, 5))
                      
    C = ax.pcolormesh(timeconv_plot, lev_sl, SpEs_freq_plot2)

    ax.set_yscale('log')
    ax.set_ylabel('Pressure / HPa')
    ax.set_ylim(lev_sl_plot_max, lev_sl_plot_min) 

    ax2 = ax.twinx()
    ax2.set_ylabel('Approx Geopotential Height / Km')
    ax2.set_ylim(Zavg_sl_plot_max,Zavg_sl_plot_min)
    ax2.set_yticklabels(Zavg_sl_plottt[lev_sl_idx_plot_max:lev_sl_idx_plot_min:-5])

    ax.set_xlabel('Local Time / hrs')
    ax.set_xticks(np.linspace(0,24,13)) 
    ax.set_xticklabels(timeconv_plot_ticks[::2]) 

    ax.set_title(Crit_name +' '+ 'SpEs_freq_plot2, Lev vs LT, lat' + lat_plot_str + ',\n UT ' + times_plot2_str )
    
    ax.grid(axis='x',color = 'grey', linestyle = '-', linewidth = 0.5)
    plt.grid(axis='y',color = 'grey', linestyle = '-', linewidth = 0.5)
    
    cbar = fig.colorbar(C, pad=0.12)#,ticks=ticks[::4],format=ticker.FuncFormatter(fmt))#fraction=0.023, 
    #-------------------

    #cbar.set_label(r'Number of Occurences')

    figname= Crit_name +' '+ 'SpEs_freq_plot2: Lev vs LT, lat' + lat_plot_str + ' ' + times_plot2_str + '.png'
    plt.tight_layout()
    #plt.savefig('./Figures/' + run_name + '/SpE_Identification/Local_Time_Plots/'+ str(Metal)+'p/Occurence_freq/Lev-LT/'+str(figname), dpi = 400) #
    